In [1]:
import pandas as pd
import requests
import urllib3
import ssl
import io

In [2]:
all_countries = ['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
                 'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
                 'Sweden', 'Spain', 'Hong Kong', 'Italy', 'Singapore',
                 'Finland', 'Belgium', 'Norway', 'Israel', 'Ireland',
                 'New Zealand', 'Austria', 'Portugal', 'Eurozone',
                 'China', 'Taiwan', 'India', 'Korea', 'Brazil',
                 'Saudi Arabia', 'South Africa', 'Mexico', 'Thailand', 'Indonesia',
                 'Malaysia', 'United Arab Emirates', 'Qatar', 'Kuwait', 'Turkiye',
                 'Philippines', 'Poland', 'Chile', 'Greece', 'Peru',
                 'Hungary', 'Czechia', 'Egypt', 'Colombia', 'Argentina', 'Russia']

In [3]:
all_countries_in_oecd = ['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
                         'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
                         'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway',
                         'Israel', 'Ireland', 'New Zealand', 'Austria', 'Portugal', 'Eurozone',
                         'China', 'India', 'Korea', 'Brazil', 'Saudi Arabia', 'South Africa',
                         'Mexico', 'Indonesia', 'Turkiye', 'Poland', 'Chile', 'Greece',
                         'Hungary', 'Czechia', 'Colombia', 'Argentina', 'Russia']

In [4]:
def format_oecd_data(df):
    df_formatted = pd.DataFrame(index=df['TIME'].unique())

    for country in df['Country'].unique():
        df_formatted[country] = df[df['Country'] == country]['Value']

    df_formatted.rename(columns={
        'Czech Republic': 'Czechia',
        'Türkiye': 'Turkiye',
        "China (People's Republic of)": 'China',
        'Euro area (19 countries)': 'Eurozone'
    }, inplace=True)

    df_formatted = df_formatted.reindex(columns=all_countries_in_oecd)
    return df_formatted

In [5]:
def check_data_coverage(df):
    available_countries = df.columns[~df.isna().all()]
    print('Available countries:')
    print(available_countries)
    print()

    no_data = df.columns[df.isna().all()]
    print('No data:')
    print(no_data)
    print()

    print('Missing first:')
    for country in df.columns:
        first_valid = df[country].first_valid_index()
        if first_valid != None and (not str(first_valid).startswith('1999')):
            print(country, first_valid)
    print()

    print('Missing last:')
    for country in df.columns:
        last_valid = df[country].last_valid_index()
        if last_valid != None and (not str(last_valid).startswith('2023')):
            print(country, last_valid)
            
    print()
    print('Done')

In [6]:
class CustomHttpAdapter (requests.adapters.HTTPAdapter):
    # "Transport adapter" that allows us to use custom ssl_context.

    def __init__(self, ssl_context=None, **kwargs):
        self.ssl_context = ssl_context
        super().__init__(**kwargs)

    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = urllib3.poolmanager.PoolManager(
            num_pools=connections, maxsize=maxsize,
            block=block, ssl_context=self.ssl_context)


def get_legacy_session():
    ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
    ctx.options |= 0x4  # OP_LEGACY_SERVER_CONNECT
    session = requests.session()
    session.mount('https://', CustomHttpAdapter(ctx))
    return session

In [7]:
countries = 'AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19'

def get_oecd_data(indicator, subject, measure, freq, file_name, 
                  countries_before_subject=False, with_measure=True,
                  filter_measure_on_df=False):
    if freq == 'Q':
        startTime = '1999-Q1'
    else:
        startTime = '1999-01'

    if countries_before_subject:
        countries_and_subject = f'{countries}.{subject}'
    else:
        countries_and_subject = f'{subject}.{countries}'
    
    if with_measure:
        url = f'https://stats.oecd.org/SDMX-JSON/data/{indicator}/{countries_and_subject}.{measure}.{freq}/all?startTime={startTime}&contentType=csv'
    else:
        url = f'https://stats.oecd.org/SDMX-JSON/data/{indicator}/{countries_and_subject}.{freq}/all?startTime={startTime}&contentType=csv'

    print(url)
    res = get_legacy_session().get(url)
    df = pd.read_csv(io.BytesIO(res.content), sep=',')
    
    if filter_measure_on_df:
        df = df[df['MEASURE'] == measure]

    df.index = df['TIME']
    
    df_formatted = format_oecd_data(df)
    df_formatted = df_formatted.sort_index()
    df_formatted = df_formatted.round(4)
    check_data_coverage(df_formatted)
    df_formatted.to_csv(f'Indicators/OECD_{file_name}.csv')
    return df_formatted

## Composite Leading Indicators

In [9]:
indicator = 'MEI_CLI' # Composite Leading Indicators (MEI) 
subject = 'BSCICP03' # OECD Standardised Bussiness Confidence Indicator, Amplitude adjusted (Long term average=100), sa
measure = ''
freq = 'M'
df = get_oecd_data(indicator, subject, measure, freq, with_measure=False, 
                   file_name='Bussiness Confidence Indicator')
df

https://stats.oecd.org/SDMX-JSON/data/MEI_CLI/BSCICP03.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'Eurozone', 'China',
       'India', 'Korea', 'Brazil', 'South Africa', 'Mexico', 'Indonesia',
       'Turkiye', 'Poland', 'Chile', 'Greece', 'Hungary', 'Czechia',
       'Colombia', 'Russia'],
      dtype='object')

No data:
Index(['Saudi Arabia', 'Argentina'], dtype='object')

Missing first:
China 2000-02
India 2000-05
Indonesia 2002-03
Chile 2003-11

Missing last:
Russia 2022-01

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,99.2362,96.9315,96.7107,100.0096,99.9062,99.0560,99.0714,99.8866,100.2446,96.2412,...,NaN,109.5673,97.5530,NaN,101.1007,98.8284,94.7566,90.5754,NaN,99.7165
1999-02,99.5429,97.0515,97.0072,100.1415,99.8676,99.2692,98.8934,99.7992,100.2559,96.5072,...,NaN,107.8086,97.4872,NaN,100.9420,98.6929,94.9350,90.3876,NaN,100.1547
1999-03,99.7711,97.1945,97.5476,100.2698,99.7866,99.3266,98.7918,99.9393,100.2438,96.7579,...,NaN,105.4042,97.5241,NaN,100.9923,98.4499,95.0126,90.5440,NaN,100.6824
1999-04,99.9549,97.3522,98.3638,100.4307,99.8711,99.4161,98.7625,100.1850,100.2430,96.9267,...,NaN,103.5620,97.6790,NaN,101.2363,97.8457,95.7322,91.2461,NaN,101.1785
1999-05,100.1645,97.5198,99.1322,100.6524,100.0901,99.4923,98.8079,100.3774,100.2582,97.1716,...,NaN,102.6633,97.7229,NaN,101.2704,97.4446,96.7618,92.1670,NaN,101.7199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05,98.8874,100.3416,99.5048,NaN,99.8476,99.9030,100.1949,99.8145,100.5179,100.0144,...,99.0410,100.0718,98.3432,95.5086,101.7147,99.0852,98.0569,99.4632,NaN,NaN
2023-06,98.8720,100.3965,99.8060,NaN,99.7915,99.7982,99.7738,NaN,100.3411,100.3803,...,99.0546,99.6269,98.2955,95.9383,101.7594,98.9231,97.3625,99.4967,NaN,NaN
2023-07,98.9092,100.4378,100.1767,NaN,99.6696,99.6981,99.3894,NaN,100.1597,100.3844,...,NaN,99.1025,98.3462,96.4850,101.6714,98.8929,97.2111,99.6655,NaN,NaN
2023-08,99.0059,100.4672,99.9707,NaN,99.6382,99.6073,99.1513,NaN,99.9948,100.0895,...,NaN,98.4377,98.5128,96.8952,101.5206,98.8091,97.3602,99.9272,NaN,NaN


In [10]:
subject = 'CSCICP03' # OECD Standardised Consumer Confidence Indicator, Amplitude adjusted (Long term average=100), sa
df = get_oecd_data(indicator, subject, measure, freq, with_measure=False, 
                   file_name='Consumer Confidence Indicator')
df

https://stats.oecd.org/SDMX-JSON/data/MEI_CLI/CSCICP03.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'France', 'Switzerland',
       'Germany', 'Australia', 'Netherlands', 'Denmark', 'Sweden', 'Spain',
       'Italy', 'Finland', 'Belgium', 'Israel', 'Ireland', 'New Zealand',
       'Austria', 'Portugal', 'Eurozone', 'China', 'India', 'Korea', 'Brazil',
       'South Africa', 'Mexico', 'Indonesia', 'Turkiye', 'Poland', 'Chile',
       'Greece', 'Hungary', 'Czechia', 'Colombia', 'Russia'],
      dtype='object')

No data:
Index(['Canada', 'Norway', 'Saudi Arabia', 'Argentina'], dtype='object')

Missing first:
Israel 2011-03
India 2012-08
Mexico 2001-04
Indonesia 2001-04
Turkiye 2004-01
Poland 2001-05
Chile 2002-03
Colombia 2001-11

Missing last:
India 2020-09

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,102.1616,99.1387,101.7730,NaN,101.2572,101.1827,100.6068,101.2532,100.9710,99.3067,...,NaN,NaN,NaN,NaN,102.4175,100.5840,96.8140,NaN,NaN,90.7300
1999-02,102.3725,99.2262,102.1197,NaN,101.2168,101.0141,100.4978,101.5505,100.8927,99.3947,...,NaN,NaN,NaN,NaN,102.4525,100.5667,96.4586,NaN,NaN,91.1700
1999-03,102.4083,99.2546,102.4171,NaN,101.0978,100.8968,100.2831,101.5727,100.8703,99.5680,...,NaN,NaN,NaN,NaN,102.5293,100.3548,96.2531,NaN,NaN,91.5000
1999-04,102.4034,99.2785,102.5353,NaN,101.0083,100.8350,100.0580,101.5487,100.9669,99.7162,...,NaN,NaN,NaN,NaN,102.7561,100.0128,96.3127,NaN,NaN,91.7200
1999-05,102.4475,99.3336,102.5404,NaN,101.0487,100.8528,99.9122,101.4571,101.0927,99.8776,...,NaN,NaN,NaN,NaN,102.9214,99.8066,96.3974,NaN,NaN,91.8700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05,97.2122,98.4040,97.1058,NaN,97.7939,96.8113,99.0019,97.7768,98.3363,98.6162,...,101.6986,99.7900,100.0494,95.7984,99.5673,98.6890,98.3285,95.8250,NaN,99.6145
2023-06,97.5081,98.5439,97.5460,NaN,98.2174,96.8765,98.9874,97.7259,98.3026,98.8779,...,101.6676,98.8301,100.5409,96.0541,100.0501,98.7942,98.4909,96.4110,NaN,NaN
2023-07,97.8740,98.5840,97.5800,NaN,98.4144,96.9681,98.8889,97.7492,98.3531,98.9324,...,101.5798,97.0670,100.8414,96.2122,100.0851,98.8955,98.7750,96.5746,NaN,NaN
2023-08,98.0031,98.4689,97.9730,NaN,98.3726,97.0720,98.7088,97.7604,98.4496,98.8330,...,101.5315,95.1146,100.8972,96.2144,99.5109,99.0041,98.8232,96.5394,NaN,NaN


In [11]:
subject = 'LOLITOAA' # Amplitude adjusted (CLI)
df = get_oecd_data(indicator, subject, measure, freq, with_measure=False, 
                   file_name='Composite Leading Indicator')
df

https://stats.oecd.org/SDMX-JSON/data/MEI_CLI/LOLITOAA.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Germany', 'Australia', 'Spain', 'Italy', 'China', 'India', 'Korea',
       'Brazil', 'South Africa', 'Mexico', 'Indonesia', 'Turkiye'],
      dtype='object')

No data:
Index(['Switzerland', 'Netherlands', 'Denmark', 'Sweden', 'Finland', 'Belgium',
       'Norway', 'Israel', 'Ireland', 'New Zealand', 'Austria', 'Portugal',
       'Eurozone', 'Saudi Arabia', 'Poland', 'Chile', 'Greece', 'Hungary',
       'Czechia', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

Missing first:

Missing last:

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,100.2252,98.0818,99.9257,99.0454,99.8682,NaN,99.3239,100.3409,NaN,NaN,...,93.4448,93.9214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-02,100.4364,98.4832,100.4169,99.3210,99.8280,NaN,99.2959,100.4413,NaN,NaN,...,94.2846,94.9234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-03,100.6263,98.8809,100.9147,99.5541,99.8623,NaN,99.3253,100.5256,NaN,NaN,...,95.1967,95.9653,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-04,100.7925,99.2591,101.3412,99.7298,99.9998,NaN,99.4127,100.6025,NaN,NaN,...,96.2100,96.8548,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-05,100.9350,99.5978,101.6694,99.8541,100.2285,NaN,99.5673,100.6697,NaN,NaN,...,97.2800,97.4931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05,98.9856,100.1302,99.5356,97.5791,98.5743,NaN,99.5351,98.7629,NaN,NaN,...,97.8906,100.3755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06,99.1008,100.1705,99.7888,97.5911,98.5674,NaN,99.3643,98.6829,NaN,NaN,...,97.9479,100.2579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07,99.2184,100.2059,100.0083,97.6391,98.5758,NaN,99.1862,98.6165,NaN,NaN,...,97.9816,100.0583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-08,99.3293,100.2406,100.2296,97.7112,98.5863,NaN,99.0290,98.5700,NaN,NaN,...,97.9914,99.8114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Key Short-Term Economic Indicators

In [11]:
indicator = 'KEI' # Key Short-Term Economic Indicators
subject = 'IRSTCI01' # Overnight Interbank Rate
measure = 'ST' # Level, ratio or index
freq = 'M' # Monthly
df = get_oecd_data(indicator, subject, measure, freq, 
                   file_name='Overnight Interbank Rate')
df

https://stats.oecd.org/SDMX-JSON/data/KEI/IRSTCI01.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.ST.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'Eurozone', 'China',
       'India', 'Korea', 'Brazil', 'South Africa', 'Mexico', 'Indonesia',
       'Turkiye', 'Poland', 'Chile', 'Greece', 'Hungary', 'Czechia',
       'Colombia', 'Russia'],
      dtype='object')

No data:
Index(['Saudi Arabia', 'Argentina'], dtype='object')

Missing first:

Missing last:
Sweden 2020-10
Russia 2022-08

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,4.63,0.2253,5.9922,4.9914,3.1370,0.8438,3.1370,4.75,3.1370,3.7699,...,36.43,78.88,15.50,12.94,11.4251,17.9000,9.09,26.98,NaN,60.0
1999-02,4.76,0.1763,5.7693,4.9957,3.1180,1.0804,3.1180,4.75,3.1180,3.5736,...,37.50,77.38,15.50,6.25,10.1807,14.8000,8.00,23.05,NaN,60.0
1999-03,4.81,0.0409,5.5665,4.9943,2.9252,1.0071,2.9252,4.75,2.9252,3.4139,...,37.84,76.32,15.50,6.84,10.1474,15.6000,7.70,22.14,NaN,60.0
1999-04,4.74,0.0314,5.3295,4.7771,2.7091,0.8036,2.7091,4.75,2.7091,3.0343,...,35.19,76.96,15.50,12.81,10.1514,15.3000,7.27,20.72,NaN,60.0
1999-05,4.74,0.0300,5.2689,4.5929,2.5510,0.8405,2.5510,4.75,2.5510,2.8990,...,28.73,76.96,15.50,11.77,10.4029,14.6000,6.91,18.92,NaN,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05,5.06,-0.0510,4.3528,4.4926,3.0463,0.9500,3.0463,3.83,3.0463,2.9000,...,5.56,NaN,6.74,11.25,3.0463,17.3508,7.00,13.28,NaN,NaN
2023-06,5.08,-0.0660,4.5872,4.6839,3.2046,1.2000,3.2046,4.05,3.2046,3.0900,...,5.61,NaN,6.71,11.25,3.2046,16.0434,7.00,13.28,NaN,NaN
2023-07,5.12,-0.0460,4.9302,4.9039,3.4033,1.2000,3.4033,4.10,3.4033,3.2300,...,5.61,NaN,6.79,11.25,3.4033,15.4338,7.00,13.26,NaN,NaN
2023-08,5.33,-0.0640,5.1609,4.9942,3.6104,1.2000,3.6104,4.10,3.6104,3.4700,...,5.62,NaN,6.66,10.25,3.6104,14.4524,7.00,13.27,NaN,NaN


In [12]:
indicator = 'KEI' # Key Short-Term Economic Indicators
subject = 'IR3TIB01' # Short term interest rate
measure = 'ST' # Level, ratio or index
freq = 'M' # Monthly
df = get_oecd_data(indicator, subject, measure, freq, 
                   file_name='Short term interest rate')
df

https://stats.oecd.org/SDMX-JSON/data/KEI/IR3TIB01.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.ST.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'Eurozone', 'China',
       'India', 'Korea', 'South Africa', 'Mexico', 'Indonesia', 'Turkiye',
       'Poland', 'Chile', 'Greece', 'Hungary', 'Czechia', 'Colombia',
       'Russia'],
      dtype='object')

No data:
Index(['Brazil', 'Saudi Arabia', 'Argentina'], dtype='object')

Missing first:
Japan 2002-04
India 2011-12

Missing last:
Turkiye 2008-04
Russia 2022-02

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,4.89,NaN,5.9217,5.0045,3.1321,NaN,3.1321,4.79,3.1321,3.8875,...,45.50,NaN,12.60,12.75,11.6900,15.8000,8.49,33.23,NaN,43.9
1999-02,4.90,NaN,5.5320,5.0335,3.0934,NaN,3.0934,4.77,3.0934,3.6150,...,38.20,NaN,12.68,10.04,10.6642,15.3000,8.18,30.25,NaN,23.3
1999-03,4.91,NaN,5.4229,5.0574,3.0467,NaN,3.0467,4.81,3.0467,3.4590,...,34.85,NaN,12.84,11.09,10.0505,15.3000,7.58,25.32,NaN,21.6
1999-04,4.88,NaN,5.3177,4.7167,2.6965,NaN,2.6965,4.76,2.6965,3.1598,...,34.09,NaN,12.95,13.59,10.0845,15.4000,7.09,22.08,NaN,20.6
1999-05,4.92,NaN,5.3488,4.6155,2.5790,NaN,2.5790,4.86,2.5790,3.1085,...,31.20,NaN,13.04,11.71,9.9450,14.9000,6.96,19.01,NaN,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05,5.15,-0.019,4.7300,4.7885,3.3664,1.45,3.3664,3.90,3.3664,3.3961,...,6.76,NaN,6.90,NaN,3.3664,14.8287,7.18,12.57,NaN,NaN
2023-06,5.22,-0.003,5.0900,5.0218,3.5359,1.45,3.5359,4.25,3.5359,3.5624,...,6.75,NaN,6.90,NaN,3.5359,15.9426,7.15,13.02,NaN,NaN
2023-07,5.35,-0.003,5.4900,5.2193,3.6718,1.72,3.6718,4.30,3.6718,3.7208,...,6.75,NaN,6.76,NaN,3.6718,10.3116,7.11,13.58,NaN,NaN
2023-08,5.44,-0.003,5.5300,5.2275,3.7803,1.77,3.7803,4.15,3.7803,3.8014,...,6.74,NaN,6.69,NaN,3.7803,13.9097,7.10,13.71,NaN,NaN


In [13]:
indicator = 'KEI' # Key Short-Term Economic Indicators
subject = 'IRLTLT01' # Long-term interest rate
measure = 'ST' # Level, ratio or index
freq = 'M' # Monthly
df = get_oecd_data(indicator, subject, measure, freq, 
                   file_name='Long term interest rate')
df

https://stats.oecd.org/SDMX-JSON/data/KEI/IRLTLT01.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.ST.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'Eurozone', 'China',
       'India', 'Korea', 'Brazil', 'South Africa', 'Mexico', 'Indonesia',
       'Poland', 'Chile', 'Greece', 'Hungary', 'Czechia', 'Colombia',
       'Russia'],
      dtype='object')

No data:
Index(['Saudi Arabia', 'Turkiye', 'Argentina'], dtype='object')

Missing first:
China 2014-01
India 2011-12
Korea 2000-10
Mexico 2001-07
Indonesia 2012-01
Poland 2001-01
Chile 2004-07
Czechia 2000-04
Colombia 2003-01

Missing last:
Russi

,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,4.72,1.910,4.2868,4.9465,3.7700,2.555,3.7020,5.1279,3.801,4.035,...,NaN,NaN,NaN,NaN,6.30,NaN,NaN,NaN,NaN,96.34
1999-02,5.00,2.117,4.4319,5.1240,3.9290,2.550,3.8490,5.3000,3.932,4.188,...,NaN,NaN,NaN,NaN,6.00,9.27,NaN,NaN,NaN,99.47
1999-03,5.23,1.816,4.6554,5.2204,4.1335,2.604,4.0378,5.5265,4.139,4.432,...,NaN,NaN,NaN,NaN,6.00,9.72,NaN,NaN,NaN,98.67
1999-04,5.18,1.563,4.5920,5.0295,3.9762,2.534,3.8500,5.3575,3.995,4.217,...,NaN,NaN,NaN,NaN,5.90,9.73,NaN,NaN,NaN,110.55
1999-05,5.54,1.334,4.9068,5.3595,4.1600,2.720,4.0121,5.8757,4.193,4.461,...,NaN,NaN,NaN,NaN,5.80,9.72,NaN,NaN,NaN,96.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05,3.57,0.430,3.9621,3.0455,2.9400,0.850,2.3423,3.5000,2.728,2.539,...,6.36,NaN,5.93,5.5486,3.98,7.78,4.50,11.11,NaN,NaN
2023-06,3.75,0.395,4.3659,3.3382,2.9300,0.970,2.3764,3.9200,2.750,2.571,...,6.22,NaN,5.91,5.3055,3.70,7.20,4.35,10.15,NaN,NaN
2023-07,3.90,0.595,4.4372,3.4520,3.0400,0.980,2.4567,4.0300,2.840,2.695,...,6.33,NaN,5.56,5.3305,3.85,7.10,4.15,10.22,NaN,NaN
2023-08,4.17,0.640,4.5298,3.6541,3.1100,0.950,2.5457,4.1300,2.917,2.768,...,6.36,NaN,5.59,5.6347,3.90,7.40,4.26,10.70,NaN,NaN


In [14]:
indicator = 'KEI' # Key Short-Term Economic Indicators
subject = 'LRHUTTTT' # Monthly Unemployment Rate - all persons, s.a.
measure = 'ST' # Level, ratio or index
freq = 'M' # Monthly
df = get_oecd_data(indicator, subject, measure, freq, 
                   file_name='Monthly Unemployment')
df.sort_index()

https://stats.oecd.org/SDMX-JSON/data/KEI/LRHUTTTT.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.ST.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Germany', 'Australia', 'Netherlands', 'Denmark', 'Sweden', 'Spain',
       'Italy', 'Finland', 'Belgium', 'Norway', 'Israel', 'Ireland', 'Austria',
       'Portugal', 'Korea', 'Mexico', 'Turkiye', 'Poland', 'Chile', 'Greece',
       'Hungary', 'Czechia', 'Colombia'],
      dtype='object')

No data:
Index(['Switzerland', 'New Zealand', 'Eurozone', 'China', 'India', 'Brazil',
       'Saudi Arabia', 'South Africa', 'Indonesia', 'Argentina', 'Russia'],
      dtype='object')

Missing first:
Israel 2012-01
Turkiye 2005-01
Colombia 2007-01

Missing last:

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,4.3,4.5,6.2,7.9,12.0,NaN,8.9,7.1187,4.6,5.0,...,NaN,NaN,11.5,8.3715,11.4,7.2,7.8,NaN,NaN,NaN
1999-02,4.4,4.6,6.2,7.9,12.0,NaN,8.8,7.1152,4.5,5.0,...,NaN,NaN,12.0,8.8078,11.4,7.2,8.1,NaN,NaN,NaN
1999-03,4.2,4.7,6.2,7.9,11.8,NaN,8.8,7.0127,4.4,5.0,...,NaN,NaN,12.3,9.3096,11.4,7.1,8.6,NaN,NaN,NaN
1999-04,4.3,4.7,6.1,8.2,11.7,NaN,8.7,6.9794,4.4,5.0,...,NaN,NaN,12.7,10.0379,12.1,7.0,8.6,NaN,NaN,NaN
1999-05,4.2,4.7,6.0,7.9,11.5,NaN,8.6,7.0250,4.3,5.0,...,NaN,NaN,12.9,10.6506,12.0,6.9,8.7,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,3.4,2.6,4.0,5.0,7.3,NaN,2.9,3.6900,3.4,4.8,...,NaN,10.0,2.8,8.2956,11.6,3.9,2.7,10.2844,NaN,NaN
2023-05,3.7,2.6,4.2,5.2,7.3,NaN,3.0,3.5708,3.5,4.8,...,NaN,9.5,2.8,8.2639,11.2,3.9,2.5,10.0911,NaN,NaN
2023-06,3.6,2.5,4.3,5.4,7.3,NaN,3.0,3.4769,3.5,4.8,...,NaN,9.6,2.8,8.3853,11.0,3.9,2.7,9.3598,NaN,NaN
2023-07,3.5,2.7,NaN,5.5,7.4,NaN,3.0,3.7238,3.6,5.6,...,NaN,9.4,2.8,8.6479,10.9,4.0,2.7,9.3144,NaN,NaN


In [15]:
indicator = 'KEI' # Key Short-Term Economic Indicators
subject = 'LRHUTTTT' # Quaterly Unemployment Rate - all persons, s.a.
measure = 'ST' # Level, ratio or index
freq = 'Q' # Quaterly
df = get_oecd_data(indicator, subject, measure, freq, 
                   file_name='Quaterly Unemployment')
df.sort_index()

https://stats.oecd.org/SDMX-JSON/data/KEI/LRHUTTTT.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.ST.Q/all?startTime=1999-Q1&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'Korea', 'Mexico',
       'Turkiye', 'Poland', 'Chile', 'Greece', 'Hungary', 'Czechia',
       'Colombia'],
      dtype='object')

No data:
Index(['Eurozone', 'China', 'India', 'Brazil', 'Saudi Arabia', 'South Africa',
       'Indonesia', 'Argentina', 'Russia'],
      dtype='object')

Missing first:
Turkiye 2005-Q1
Colombia 2007-Q1

Missing last:

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-Q1,4.3000,4.6000,6.2,7.9000,11.9333,NaN,8.8333,7.0822,4.5000,5.0000,...,NaN,NaN,11.9333,8.8078,11.4000,7.1667,8.1667,NaN,NaN,NaN
1999-Q2,4.2667,4.7333,6.0,7.9000,11.5333,3.0992,8.6333,6.8860,4.3333,4.9667,...,NaN,NaN,12.9333,10.6506,12.0333,6.9333,8.6000,NaN,NaN,NaN
1999-Q3,4.2333,4.7000,5.9,7.5000,11.1333,NaN,8.4667,6.8741,4.1667,5.1667,...,NaN,NaN,13.8333,10.9703,12.0333,6.7333,8.9333,NaN,NaN,NaN
1999-Q4,4.0667,4.6333,5.8,6.9667,10.6667,NaN,8.3000,6.6337,4.0333,5.3000,...,NaN,NaN,14.8000,10.1521,12.3333,6.6667,9.0667,NaN,NaN,NaN
2000-Q1,4.0333,4.8333,5.8,6.8667,10.2333,NaN,8.1333,6.6505,3.8667,4.6667,...,NaN,NaN,15.8000,9.0066,11.5667,6.5667,9.2667,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-Q2,3.6000,2.6000,3.8,5.1333,7.5000,4.3511,3.0000,3.8004,3.3000,4.1333,...,NaN,10.6667,2.8000,7.5662,12.6000,3.5333,2.4667,10.7334,NaN,NaN
2022-Q3,3.5667,2.5667,3.6,5.1333,7.2000,4.1760,3.0667,3.5038,3.7333,4.5667,...,NaN,10.0333,2.9333,7.8100,12.2667,3.6333,2.3667,10.9531,NaN,NaN
2022-Q4,3.6000,2.5333,3.7,5.1000,7.2000,4.2534,3.0333,3.4692,3.6000,4.7667,...,NaN,10.1333,2.9000,8.4073,11.6667,3.7667,2.4000,11.0421,NaN,NaN
2023-Q1,3.5000,2.6000,3.9,5.0000,7.1000,4.1342,2.9667,3.5788,3.5333,4.7667,...,NaN,9.9333,2.7000,8.7302,11.2667,3.8000,2.5333,10.6234,NaN,NaN


In [16]:
indicator = 'KEI' # Key Short-Term Economic Indicators
subject = 'CPALTT01' # Consumer Prices - Annual inflation
measure = 'GY' # Growth on the same period of the previous year
freq = 'M' # Monthly
df = get_oecd_data(indicator, subject, measure, freq, 
                   'Inflation Rate')
df

https://stats.oecd.org/SDMX-JSON/data/KEI/CPALTT01.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.GY.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Netherlands', 'Denmark', 'Sweden', 'Spain',
       'Italy', 'Finland', 'Belgium', 'Norway', 'Israel', 'Ireland', 'Austria',
       'Portugal', 'China', 'India', 'Korea', 'Brazil', 'Saudi Arabia',
       'South Africa', 'Mexico', 'Indonesia', 'Turkiye', 'Poland', 'Chile',
       'Greece', 'Hungary', 'Czechia', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

No data:
Index(['Australia', 'New Zealand', 'Eurozone'], dtype='object')

Missing first:
Argentina 2017-12

Missing last:
Japan 2021-06
Russia 2022-03

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,1.6708,0.2,2.0,0.6601,0.2443,0.0729,0.2389,NaN,2.1779,1.7040,...,70.6425,65.9017,6.9,3.5949,3.6965,9.7531,3.5874,17.1793,NaN,96.9186
1999-02,1.6059,-0.1,1.9,0.6593,0.1793,0.2880,0.1192,NaN,2.1094,2.0359,...,53.4074,63.9271,5.6,3.8041,3.6893,9.3949,2.9718,15.3797,NaN,103.2425
1999-03,1.7263,-0.4,2.1,0.9879,0.3453,0.4670,0.2387,NaN,2.1478,2.3222,...,45.4167,63.5374,6.2,4.0523,3.3996,9.2749,2.6667,13.4950,NaN,107.5844
1999-04,2.2769,-0.1,2.0,1.6484,0.4211,0.5938,0.5959,NaN,2.1146,2.3649,...,37.9676,63.8550,6.3,4.0334,2.8060,9.3932,2.5111,11.1604,NaN,113.0646
1999-05,2.0885,-0.4,1.7,1.5334,0.4083,0.5758,0.4762,NaN,2.2420,2.0653,...,30.7363,62.9667,6.4,3.9669,2.3726,8.8637,2.5111,9.9980,NaN,116.7111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05,4.0476,NaN,7.9,3.3575,5.1199,2.1587,6.1020,NaN,6.0563,2.9281,...,4.0042,39.5853,13.0,8.7268,2.7815,21.5000,11.0609,12.3603,114.2463,NaN
2023-06,2.9692,NaN,7.3,2.8123,4.5313,1.7109,6.3752,NaN,5.6951,2.4648,...,3.5210,38.2135,11.5,7.5604,1.7605,20.1000,9.7037,12.1296,115.5803,NaN
2023-07,3.1778,NaN,6.4,3.2658,4.2881,1.6051,6.1650,NaN,4.5735,3.1332,...,3.0782,47.8317,10.8,6.4755,2.4708,17.6000,8.7719,11.7849,113.4500,NaN
2023-08,3.6651,NaN,6.3,3.9974,4.8598,1.5543,6.1427,NaN,3.0254,2.4369,...,3.2728,58.9435,10.1,5.3155,2.7441,16.4000,8.5153,11.4273,124.3675,NaN


In [17]:
indicator = 'KEI' # Key Short-Term Economic Indicators
subject = 'CPALTT01' # Consumer Prices - Annual inflation
measure = 'GP' # Growth previous period
freq = 'M' # Monthly
df = get_oecd_data(indicator, subject, measure, freq, 
                   'Inflation Rate MoM')
df

https://stats.oecd.org/SDMX-JSON/data/KEI/CPALTT01.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.GP.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Netherlands', 'Denmark', 'Sweden', 'Spain',
       'Italy', 'Finland', 'Belgium', 'Norway', 'Israel', 'Ireland', 'Austria',
       'Portugal', 'China', 'India', 'Korea', 'Brazil', 'Saudi Arabia',
       'South Africa', 'Mexico', 'Indonesia', 'Turkiye', 'Poland', 'Chile',
       'Greece', 'Hungary', 'Czechia', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

No data:
Index(['Australia', 'New Zealand', 'Eurozone'], dtype='object')

Missing first:
Argentina 2017-01

Missing last:
Japan 2021-06
Russia 2022-03

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,0.2441,-0.5,-0.4,0.2191,-0.2814,0.2050,-0.1190,NaN,0.0554,-0.0956,...,2.9622,4.8028,1.5,-0.3328,-1.0097,2.5137,0.7267,2.2231,NaN,8.38
1999-02,0.1217,-0.4,0.2,0.1093,0.3078,0.3380,0.1192,NaN,0.4983,0.7659,...,1.2744,3.1731,0.6,0.0720,-0.4827,1.3274,0.0000,1.6873,NaN,4.13
1999-03,0.3040,0.1,0.5,0.4367,0.3324,0.1046,0.0000,NaN,0.8676,0.4751,...,-0.1907,4.0541,1.0,0.6357,2.2150,1.1827,0.0000,0.9403,NaN,2.79
1999-04,0.7273,0.5,0.5,0.5435,0.2804,0.1668,0.4762,NaN,0.2185,0.2837,...,-0.6686,4.8672,0.8,0.3759,1.0403,1.1509,0.1443,0.7854,NaN,3.03
1999-05,0.0000,0.0,0.2,0.2162,0.0127,-0.2243,0.0000,NaN,0.0272,0.1886,...,-0.2885,2.9408,0.7,0.1182,-0.0520,0.6578,0.0000,0.4893,NaN,2.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05,0.2518,NaN,0.6,0.3836,-0.0511,0.2510,-0.0858,NaN,0.1984,-1.3605,...,0.0872,0.0431,0.0,0.1138,0.4031,-0.4000,0.2717,0.4312,7.7728,NaN
2023-06,0.3229,NaN,0.2,0.1274,0.1788,0.0636,0.2575,NaN,-0.3406,0.3448,...,0.1393,3.9205,0.0,-0.1516,0.6186,0.3000,0.3388,0.3009,5.9508,NaN
2023-07,0.1908,NaN,-0.3,0.5725,0.0510,-0.1418,0.2568,NaN,1.0331,1.8041,...,0.2087,9.4888,-0.2,0.3492,-1.1451,0.3000,0.4727,0.5017,6.3449,NaN
2023-08,0.4367,NaN,0.4,0.3795,1.0025,0.2135,0.3416,NaN,0.4484,-0.6751,...,-0.0174,9.0868,0.0,0.1135,-0.0318,0.7000,0.2016,0.6986,12.4416,NaN


In [18]:
subject = 'B6BLTT02' # Current Account as a % of GDP
measure = 'ST' # Level, ratio or index
freq = 'Q'
df = get_oecd_data(indicator, subject, measure, freq, 
                   file_name='Current Account')
df

https://stats.oecd.org/SDMX-JSON/data/KEI/B6BLTT02.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.ST.Q/all?startTime=1999-Q1&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'China', 'India',
       'Korea', 'Brazil', 'Saudi Arabia', 'South Africa', 'Mexico',
       'Indonesia', 'Turkiye', 'Poland', 'Chile', 'Greece', 'Hungary',
       'Czechia', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

No data:
Index(['Eurozone'], dtype='object')

Missing first:
Netherlands 2003-Q2
Denmark 2005-Q1
Belgium 2003-Q1
Ireland 2002-Q1
Saudi Arabia 2006-Q1
Mexico 2006-Q1
Indonesia 2004-Q1
Poland 2004-Q1
Chile 2003-Q1
Greece

,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-Q1,-2.5856,2.3287,-2.7743,-0.1468,3.7575,2.0488,-1.9899,-5.6295,NaN,NaN,...,NaN,3.2567,NaN,NaN,NaN,-7.8299,-2.9088,NaN,NaN,NaN
1999-Q2,-2.8535,2.4608,-2.2026,-0.2523,3.5495,10.1592,-0.7914,-5.9356,NaN,NaN,...,NaN,-1.2266,NaN,NaN,NaN,-9.3325,-1.8839,NaN,NaN,NaN
1999-Q3,-3.1437,2.5760,-2.2103,0.7242,2.2618,11.7423,-1.3592,-5.6566,NaN,NaN,...,NaN,-0.8913,NaN,NaN,NaN,-8.3765,-0.5894,NaN,NaN,NaN
1999-Q4,-3.3003,2.5238,-1.3045,0.3600,3.7201,10.0676,-1.7366,-4.7052,NaN,NaN,...,NaN,-1.3361,NaN,NaN,NaN,-8.1374,-3.1151,NaN,NaN,NaN
2000-Q1,-3.7901,2.6196,-1.5893,2.0864,1.0562,8.5940,-1.1893,-4.8409,NaN,NaN,...,NaN,-2.7750,NaN,NaN,NaN,-8.1085,-3.7479,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-Q2,-3.8957,2.2910,-4.5342,0.6844,-1.2825,6.1799,4.0163,1.9730,6.6455,13.1557,...,0.6954,-3.3086,-2.6316,-12.4454,-11.5177,-7.1128,-6.3148,-5.9343,-1.3475,NaN
2022-Q3,-3.4277,0.7449,-2.0208,-1.3991,-3.1927,13.9123,3.9734,-0.1222,10.5008,16.2150,...,0.5759,-6.9257,-1.1011,-8.7041,-7.3942,-10.6342,-7.2217,-7.1611,-2.0619,NaN
2022-Q4,-3.2740,1.7591,-0.3851,-1.1566,-2.3918,6.8426,4.1428,1.7281,7.2824,12.8691,...,2.1953,-4.8877,-1.7206,-6.0130,-13.4611,-8.2054,-5.7903,-6.1688,0.9695,NaN
2023-Q1,-3.1995,1.9152,-1.6340,-0.4522,-1.1246,9.8778,5.3476,1.9419,11.3461,12.8315,...,1.0801,-6.4575,2.1446,0.6110,0.8651,-3.2070,-1.0528,-3.5616,-2.7089,NaN


In [19]:
subject = 'PRINTO01' # Industrial production
measure = 'GY' # Growth on the same period of the prevoius year
freq = 'M'
df = get_oecd_data(indicator, subject, measure, freq, 
                   file_name='Industrial Production')
df

https://stats.oecd.org/SDMX-JSON/data/KEI/PRINTO01.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.GY.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Netherlands', 'Denmark', 'Sweden', 'Spain',
       'Italy', 'Finland', 'Belgium', 'Norway', 'Israel', 'Ireland', 'Austria',
       'Portugal', 'Eurozone', 'India', 'Korea', 'Brazil', 'Turkiye', 'Poland',
       'Chile', 'Greece', 'Hungary', 'Czechia', 'Colombia', 'Russia'],
      dtype='object')

No data:
Index(['Australia', 'New Zealand', 'China', 'Saudi Arabia', 'South Africa',
       'Mexico', 'Indonesia', 'Argentina'],
      dtype='object')

Missing first:
Switzerland 2011-10

Missing last:
Russia 2021-12

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,3.6382,-6.2062,2.1445,5.4556,1.4157,NaN,0.4005,NaN,1.0858,1.8066,...,NaN,-5.9908,-4.1235,-5.3578,5.3072,13.4454,-8.7027,-14.4379,NaN,-2.2439
1999-02,4.0588,-3.6999,0.7839,4.0738,0.4129,NaN,-1.2048,NaN,3.1938,0.8584,...,NaN,-5.5172,-5.1703,-4.8663,2.6222,5.5480,-7.6562,-17.8520,NaN,-9.3602
1999-03,4.2431,-0.3151,1.3393,2.6733,1.5421,NaN,-1.8494,NaN,3.8377,1.0526,...,NaN,-4.9774,1.8323,-3.4553,-5.5786,8.5020,-9.6097,-22.5638,NaN,-5.8613
1999-04,4.1228,-1.7895,1.2304,3.7553,1.0311,NaN,-0.2667,NaN,0.8677,0.3337,...,NaN,-4.3880,-0.0370,-5.3748,-2.1619,8.0107,-7.0478,-19.8936,NaN,-5.7712
1999-05,4.0753,0.5302,3.6364,4.3957,0.1746,NaN,-0.1326,NaN,-0.2148,3.5437,...,NaN,-3.3860,3.2203,-3.4312,3.5668,4.4271,-1.8452,-20.1074,NaN,-1.1373
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,0.3231,0.1905,-1.3133,0.6552,1.5150,NaN,2.5316,NaN,-11.6954,10.1629,...,NaN,-0.8898,-3.4054,-1.0212,4.5061,-5.6937,1.6874,-4.7580,NaN,NaN
2023-05,0.0092,2.3000,-2.0619,0.6172,2.2382,NaN,1.6684,NaN,-8.8208,6.4955,...,NaN,0.4104,-2.3163,-5.3892,1.6849,-4.3131,0.6124,-3.4318,NaN,NaN
2023-06,-0.3107,-0.1896,0.7576,-0.0585,-0.2629,NaN,0.0000,NaN,-6.3089,11.4840,...,NaN,0.2023,-0.9352,-2.2156,-3.6408,-5.8591,0.3442,-4.8481,NaN,NaN
2023-07,-0.0396,-2.2642,0.4753,-0.2251,2.6745,NaN,-1.3528,NaN,-6.4546,4.2178,...,NaN,6.9314,-1.9333,-1.9822,-2.1687,-2.6274,-2.3216,-6.5805,NaN,NaN


In [20]:
subject = 'NAEXKP02' # Private Consumption
measure = 'GY' # Growth on the same period of the prevoius year
freq = 'Q'
df = get_oecd_data(indicator, subject, measure, freq, 
                   file_name='Private Consumption')
df

https://stats.oecd.org/SDMX-JSON/data/KEI/NAEXKP02.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.GY.Q/all?startTime=1999-Q1&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'India', 'Korea',
       'Brazil', 'South Africa', 'Mexico', 'Indonesia', 'Turkiye', 'Poland',
       'Chile', 'Greece', 'Hungary', 'Czechia', 'Colombia', 'Russia'],
      dtype='object')

No data:
Index(['Eurozone', 'China', 'Saudi Arabia', 'Argentina'], dtype='object')

Missing first:
India 2012-Q2
Colombia 2006-Q1
Russia 2004-Q1

Missing last:
Russia 2021-Q3

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-Q1,5.6513,0.8615,3.4563,3.5765,3.0230,2.1917,2.6510,6.4909,6.6409,1.7525,...,2.1890,-0.7260,4.1096,-3.0433,3.0519,6.1402,3.3655,NaN,NaN,NaN
1999-Q2,5.4130,1.0395,4.7534,3.3629,2.8210,2.0994,3.2663,5.4950,6.3220,-0.2225,...,3.8922,3.5276,6.2219,-4.9401,2.2762,6.9139,3.5238,NaN,NaN,NaN
1999-Q3,5.2259,1.0440,4.1310,4.0479,4.0008,2.0255,2.0089,4.6132,5.6900,-0.5135,...,7.6570,-1.8747,4.4846,-0.7301,3.1149,7.0728,3.6451,NaN,NaN,NaN
1999-Q4,5.2446,1.4707,4.8783,4.7435,3.8454,2.2848,1.9940,5.0556,5.0139,-1.4478,...,4.8990,-0.5397,6.6983,4.8402,1.4243,6.4830,1.9286,NaN,NaN,NaN
2000-Q1,5.7498,1.6474,5.1566,4.3436,4.7717,2.1592,1.6463,3.4777,4.7535,-0.0190,...,0.8067,3.6104,4.3640,5.0156,2.4071,4.2588,1.2688,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-Q2,2.1945,2.5592,4.8261,7.9260,4.7604,4.3314,5.2642,5.2958,7.0025,-1.4986,...,4.8809,21.0110,6.3860,7.4782,8.8296,7.8147,-0.1302,14.9106,NaN,NaN
2022-Q3,1.8760,3.7468,1.6335,3.1382,-0.0303,2.6527,1.6054,11.5479,2.8907,-3.5427,...,5.2130,18.1317,2.0243,-3.4150,5.4811,4.6043,-4.5433,9.1295,NaN,NaN
2022-Q4,1.1720,0.9557,1.6490,3.0397,-0.8834,2.9962,0.5125,5.1988,4.2371,-4.9904,...,5.0591,15.8631,-1.4123,-4.4097,3.9571,2.3702,-5.2587,3.9225,NaN,NaN
2023-Q1,2.1182,2.6174,0.2579,3.7570,0.4109,3.1191,-0.3681,3.5625,3.1735,-0.4813,...,5.0277,16.8475,-2.5628,-7.1358,2.4969,-4.4218,-5.3272,3.0551,NaN,NaN


## Quaterly National Accounts

In [21]:
indicator = 'QNA' # Quaterly National Accounts
subject = 'B1_GE' # Gross domestic product - expenditure approach
measure = 'GYSA' # Growth rate based on seasonally adjusted volume data, percentage change on the same quarter the previous year
freq = 'Q' # Quaterly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='Quaterly GDP YoY')
df

https://stats.oecd.org/SDMX-JSON/data/QNA/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.B1_GE.GYSA.Q/all?startTime=1999-Q1&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'China', 'India',
       'Korea', 'Brazil', 'Saudi Arabia', 'South Africa', 'Mexico',
       'Indonesia', 'Turkiye', 'Poland', 'Chile', 'Greece', 'Hungary',
       'Czechia', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

No data:
Index(['Eurozone'], dtype='object')

Missing first:
Saudi Arabia 2011-Q1
Colombia 2006-Q1
Russia 2004-Q1

Missing last:
Russia 2021-Q3

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-Q1,4.8163,-0.8405,2.3399,4.1941,2.7842,1.0304,0.7938,5.1738,4.2134,2.3111,...,-6.4079,-4.0900,2.1001,-2.2375,2.3617,2.5265,0.3953,NaN,-2.1765,NaN
1999-Q2,4.7219,-0.0338,2.1742,5.0097,2.6179,0.5713,1.2206,4.5412,4.6244,3.6219,...,1.4970,-3.8649,4.2241,-3.6060,2.8620,2.0658,0.5752,NaN,-5.1406,NaN
1999-Q3,4.7911,0.3310,3.4709,5.6907,3.5293,1.1757,1.8595,3.7770,5.1676,2.3813,...,2.9213,-4.1375,5.2988,-0.9069,2.8964,3.0412,1.4733,NaN,-5.1249,NaN
1999-Q4,4.8236,-0.4356,4.2619,5.7387,4.1792,3.7357,3.1243,3.9628,5.6801,3.4815,...,5.9918,-1.0176,6.5294,5.1053,3.7243,4.1765,2.5429,NaN,-1.0144,NaN
2000-Q1,4.2247,2.6673,4.8207,5.5164,4.3571,4.3203,3.3992,3.6236,4.5949,3.5523,...,3.4834,2.2652,6.3778,6.0779,3.3304,5.1371,3.8794,NaN,-0.1064,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-Q2,1.8706,1.4347,3.8968,4.7197,3.8758,3.2883,1.5552,3.0926,5.1520,3.1481,...,5.2889,6.9877,5.9779,5.4998,7.1231,6.3513,3.3596,12.1518,7.6889,NaN
2022-Q3,1.7132,1.5209,2.0570,3.8189,1.1804,1.3772,1.2457,5.9941,2.9922,2.2440,...,6.1114,3.7823,5.0068,0.0072,4.0506,3.9757,1.3501,7.3321,5.3425,NaN
2022-Q4,0.6517,0.4618,0.6634,2.0741,0.7194,1.1535,0.8250,2.6965,3.0649,0.3771,...,5.1579,3.3548,0.9905,-2.1371,4.7541,0.5170,0.1200,1.9704,1.1023,NaN
2023-Q1,1.7179,1.8496,0.4584,2.0793,0.8263,1.4434,-0.2684,2.4445,2.0982,2.2555,...,4.9402,3.0054,-0.6183,-0.9576,1.9689,-1.1514,-0.3778,2.9986,1.2259,NaN


In [22]:
indicator = 'QNA' # Quaterly National Accounts
subject = 'B1_GE' # Gross domestic product - expenditure approach
measure = 'GPSA' # Growth rate based on seasonally adjusted volume data, percentage change on the same quarter the previous year
freq = 'Q' # Quaterly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='Quaterly GDP QoQ')
df

https://stats.oecd.org/SDMX-JSON/data/QNA/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.B1_GE.GPSA.Q/all?startTime=1999-Q1&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'China', 'India',
       'Korea', 'Brazil', 'Saudi Arabia', 'South Africa', 'Mexico',
       'Indonesia', 'Turkiye', 'Poland', 'Chile', 'Greece', 'Hungary',
       'Czechia', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

No data:
Index(['Eurozone'], dtype='object')

Missing first:
China 2011-Q1
Saudi Arabia 2010-Q2
Colombia 2005-Q2
Russia 2003-Q2

Missing last:
Russia 2021-Q3

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-Q1,0.9394,-1.3823,0.6929,1.8138,0.6634,0.1031,1.1576,0.7300,1.6668,0.9060,...,5.6477,-1.6858,0.8045,0.0192,0.7623,-0.0074,0.0263,NaN,-1.2459,NaN
1999-Q2,0.8347,0.3748,0.4744,0.8365,0.6261,0.6738,0.0000,0.3249,1.1129,0.5164,...,-0.0550,-0.5637,2.0946,0.2728,0.9332,1.0976,0.4539,NaN,-1.1299,NaN
1999-Q3,1.3259,0.5443,1.5998,1.5647,1.4696,0.5661,1.1074,1.1585,1.3772,0.6181,...,0.2734,-1.6866,2.0008,2.1095,0.3828,1.8885,0.9237,NaN,-0.2882,NaN
1999-Q4,1.6407,0.0385,1.4329,1.4067,1.3591,2.3559,0.8276,1.6969,1.4069,1.3988,...,0.1072,2.9874,1.4807,2.6341,1.5989,1.1430,1.1188,NaN,1.6728,NaN
2000-Q1,0.3628,1.6912,1.2325,1.5997,0.8353,0.6672,1.4274,0.4014,0.6227,0.9751,...,3.1475,1.5749,0.6611,0.9448,0.3797,0.9146,1.3299,NaN,-0.3400,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-Q2,-0.1413,1.2918,0.0892,0.8970,0.3792,0.8709,-0.1296,0.7233,1.8136,1.0253,...,0.9127,1.4144,-1.3899,-0.1661,0.5042,0.9650,0.1529,1.5884,1.7125,NaN
2022-Q3,0.6586,-0.2978,-0.0856,0.5584,0.3150,0.2629,0.3614,0.6744,-0.1741,-0.2012,...,0.8208,0.5422,1.2391,-1.2319,0.2647,-0.8790,-0.2257,0.8594,0.4534,NaN
2022-Q4,0.6354,0.0507,0.1338,-0.0372,0.1108,0.0214,-0.4063,0.6528,0.8758,0.5769,...,2.3052,1.1485,-2.0142,0.0389,1.1938,-0.9077,-0.3917,-1.6606,-1.7971,NaN
2023-Q1,0.5564,0.7999,0.3207,0.6475,0.0192,0.2824,-0.0927,0.3723,-0.4178,0.8400,...,0.8203,-0.1260,1.5952,0.4055,-0.0042,-0.3233,0.0871,2.2217,0.8854,NaN


In [23]:
indicator = 'QNA' # Quaterly National Accounts
subject = 'P6' # Export of goods and services
measure = 'GYSA' # Growth rate based on seasonally adjusted volume data, percentage change on the same quarter the previous year
freq = 'Q' # Quaterly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='Export Growth YoY')
df

https://stats.oecd.org/SDMX-JSON/data/QNA/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.P6.GYSA.Q/all?startTime=1999-Q1&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'India', 'Korea',
       'Brazil', 'South Africa', 'Mexico', 'Indonesia', 'Turkiye', 'Poland',
       'Chile', 'Greece', 'Hungary', 'Czechia', 'Colombia', 'Argentina',
       'Russia'],
      dtype='object')

No data:
Index(['Eurozone', 'China', 'Saudi Arabia'], dtype='object')

Missing first:
Colombia 2006-Q1
Argentina 2005-Q1
Russia 2004-Q1

Missing last:
Russia 2021-Q3

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-Q1,1.7671,-3.1844,-0.2738,11.1567,2.6464,-0.9712,-0.4279,3.4127,8.1007,10.0671,...,-43.7356,-6.3549,-6.3841,6.6306,27.6032,7.7228,-4.6848,NaN,NaN,NaN
1999-Q2,4.1793,-0.3341,-0.2048,10.0931,2.3475,-0.3238,3.0436,1.2034,8.5001,14.3649,...,-37.4547,-13.3143,-5.7210,7.0647,34.6068,7.8441,5.5181,NaN,NaN,NaN
1999-Q3,7.8129,3.5578,6.1862,11.6743,5.2632,6.4640,6.5276,5.6136,8.2034,6.9290,...,-38.6284,-12.6153,1.4279,6.7546,11.7046,12.2069,8.3122,NaN,NaN,NaN
1999-Q4,6.1694,7.8805,8.3080,10.4025,8.4392,7.6024,10.6199,8.5590,9.9823,14.0064,...,12.3186,-9.2350,0.5646,7.5023,26.4494,19.1028,10.8436,NaN,NaN,NaN
2000-Q1,8.0588,13.4210,9.9454,10.1258,12.3609,12.2576,15.1947,12.1008,11.0108,7.5235,...,14.7285,7.9059,20.5573,8.3427,29.1636,23.7174,16.3557,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-Q2,7.3840,2.9339,2.4642,4.5699,7.7540,-2.2730,3.7893,4.0195,5.9313,11.4278,...,18.3203,17.6857,6.5041,0.6115,12.3634,10.8111,3.3221,27.8877,8.4132,NaN
2022-Q3,11.0691,5.8722,18.9985,6.4427,7.2693,10.2295,5.4926,6.2723,2.9572,14.8102,...,18.6452,11.0552,8.4830,3.5014,-2.6570,17.4880,12.7227,16.2137,-2.6813,NaN
2022-Q4,4.2846,7.3381,8.3093,3.0765,4.1771,3.9844,0.6987,8.1837,4.6018,7.9313,...,12.5291,-3.6516,4.3267,0.6661,-2.0156,12.8537,10.8538,-0.5249,8.3348,NaN
2023-Q1,7.2763,1.8641,9.9046,7.8946,0.6974,3.7381,1.2635,10.8665,2.6181,8.4409,...,9.8040,-2.7369,2.9564,2.6592,7.1057,6.3173,6.4462,2.5718,-4.2891,NaN


In [24]:
indicator = 'QNA' # Quaterly National Accounts
subject = 'P7' # Import of goods and services
measure = 'GYSA' # Growth rate based on seasonally adjusted volume data, percentage change on the same quarter the previous year
freq = 'Q' # Quaterly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='Import Growth YoY')
df

https://stats.oecd.org/SDMX-JSON/data/QNA/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.P7.GYSA.Q/all?startTime=1999-Q1&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'India', 'Korea',
       'Brazil', 'South Africa', 'Mexico', 'Indonesia', 'Turkiye', 'Poland',
       'Chile', 'Greece', 'Hungary', 'Czechia', 'Colombia', 'Argentina',
       'Russia'],
      dtype='object')

No data:
Index(['Eurozone', 'China', 'Saudi Arabia'], dtype='object')

Missing first:
Colombia 2006-Q1
Argentina 2005-Q1
Russia 2004-Q1

Missing last:
Russia 2021-Q3

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-Q1,10.0517,-3.6590,6.2706,4.1953,5.1630,-1.6938,7.0441,4.1251,6.8348,2.1500,...,-52.5869,-16.1275,6.2487,-14.2995,11.2439,11.6826,-0.5276,NaN,NaN,NaN
1999-Q2,10.6759,2.6584,5.0500,5.5722,4.6778,2.9475,7.6493,5.8470,9.2038,1.3258,...,-44.7959,-0.2327,3.8637,-14.6244,22.1164,11.7955,5.9148,NaN,NaN,NaN
1999-Q3,13.1501,5.0853,8.0784,10.6278,6.2580,8.0020,9.5921,11.0909,9.8638,1.4103,...,-43.1295,-3.6417,-1.3991,-10.8983,14.4151,12.0979,3.7894,NaN,NaN,NaN
1999-Q4,12.4061,10.8835,10.1378,11.7996,9.4004,7.5378,10.0690,12.3678,12.0777,5.2740,...,-9.0146,7.5883,-3.5673,1.4276,11.4847,17.2570,10.8066,NaN,NaN,NaN
2000-Q1,13.4442,7.6032,8.2743,12.0462,13.3730,11.6007,9.2564,13.0296,11.3313,10.0383,...,7.3423,27.1881,13.5823,9.2879,21.7409,22.5173,13.6092,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-Q2,11.7861,3.4650,18.5454,11.3234,8.0284,1.5853,7.7623,9.9246,4.9514,7.4991,...,16.4542,7.5159,8.2004,10.8615,14.6016,10.3287,3.0401,25.5258,26.0426,NaN
2022-Q3,8.2054,10.9347,9.9051,11.5342,11.7357,13.0475,10.2089,19.4148,1.9013,7.3278,...,19.2435,13.6567,7.1438,-2.9441,5.2189,13.9119,8.0832,21.6278,21.4556,NaN
2022-Q4,2.1317,10.4467,8.1181,2.9669,5.5583,3.4715,2.5888,12.7590,4.1435,4.5719,...,5.2511,9.8458,0.9651,-15.7631,6.7865,10.8032,8.2523,7.4256,0.7655,NaN
2023-Q1,-0.9923,4.1282,-2.8480,2.6541,1.0577,9.5040,1.0111,5.6757,2.3015,2.1350,...,4.2904,14.6333,-4.0239,-16.8681,5.1639,2.2177,2.9839,-7.5915,5.2209,NaN


In [25]:
indicator = 'QNA' # Quaterly National Accounts
subject = 'B1_GE' # GDP per capita
measure = 'HVPVOBARSA' # Per Head, US dollars, volume estimates, fixed PPPs, OECD reference year (2015), seasonally adjusted
freq = 'Q' # Quaterly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='GDP per capita')
df

https://stats.oecd.org/SDMX-JSON/data/QNA/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.B1_GE.HVPVOBARSA.Q/all?startTime=1999-Q1&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'Korea',
       'Saudi Arabia', 'South Africa', 'Mexico', 'Poland', 'Chile', 'Greece',
       'Hungary', 'Czechia', 'Colombia', 'Russia'],
      dtype='object')

No data:
Index(['Eurozone', 'China', 'India', 'Brazil', 'Indonesia', 'Turkiye',
       'Argentina'],
      dtype='object')

Missing first:
Japan 2007-Q3
Saudi Arabia 2010-Q1
South Africa 2008-Q1
Mexico 2005-Q1
Colombia 2001-Q1
Russia 2003-Q1

Missing last:
United Kingdom 2022-Q4
Saudi

,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-Q1,46594.9,NaN,34884.5,36810.5,35777.0,54309.0,38826.6,36641.5,42787.7,43465.4,...,NaN,NaN,14511.0,13691.5,27526.6,17635.6,22153.5,NaN,NaN,NaN
1999-Q2,46855.7,NaN,35019.0,37051.6,35950.6,54600.7,38821.3,36678.9,43203.6,43657.0,...,NaN,NaN,14816.1,13674.3,27765.8,17843.0,22260.6,NaN,NaN,NaN
1999-Q3,47326.4,NaN,35548.3,37524.2,36423.2,54835.0,39240.6,36994.9,43715.6,43885.6,...,NaN,NaN,15114.2,13912.6,27853.9,18190.7,22464.0,NaN,NaN,NaN
1999-Q4,47958.7,NaN,36027.0,37938.7,36856.9,56050.4,39562.9,37520.2,44246.8,44466.0,...,NaN,NaN,15339.2,14231.1,28281.0,18412.9,22719.8,NaN,NaN,NaN
2000-Q1,48010.8,NaN,36439.4,38503.0,37098.4,56347.6,40121.7,37536.9,44452.2,44865.9,...,NaN,NaN,15442.2,14318.1,28359.0,18596.5,23037.6,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-Q2,63378.8,42410.2,44760.1,46352.8,42911.8,70248.5,50069.3,50282.5,56503.6,55730.8,...,NaN,NaN,35412.4,23888.4,29697.7,34565.2,38507.7,15024.7,NaN,NaN
2022-Q3,63704.9,42344.1,44713.2,46270.4,43011.2,70277.5,50119.9,50350.1,56182.7,55430.8,...,NaN,NaN,35873.0,23552.4,29811.4,34270.3,38428.0,15115.0,NaN,NaN
2022-Q4,64017.0,42431.8,44763.8,45826.5,43022.9,70109.0,49842.1,50412.8,56515.5,55665.9,...,NaN,NaN,35178.4,23522.4,30225.4,33966.9,38270.3,14824.3,NaN,NaN
2023-Q1,64304.2,42868.1,NaN,45803.9,42995.2,69890.3,49655.5,50253.2,56187.8,56039.0,...,NaN,NaN,35777.4,23578.5,30260.1,34144.4,38242.8,15113.5,NaN,NaN


### Gross domestic product - Current prices - Yearly

In [26]:
indicator = 'SNA_TABLE1' # Gross domestic product
subject = 'B1_GE' # GDP Current Prices
measure = 'VXCOB' # Current Prices, constant exchange rate, OECD base year, (US Dollar, Million, 2015)
freq = '' # Yearly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='GDP Current Prices', with_measure=False, 
                   filter_measure_on_df=True)
df

https://stats.oecd.org/SDMX-JSON/data/SNA_TABLE1/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.B1_GE./all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'Eurozone', 'China',
       'India', 'Korea', 'Brazil', 'Saudi Arabia', 'South Africa', 'Mexico',
       'Indonesia', 'Turkiye', 'Poland', 'Chile', 'Greece', 'Hungary',
       'Czechia', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

No data:
Index([], dtype='object')

Missing first:
Argentina 2004

Missing last:
United States 2022
Japan 2022
United Kingdom 2022
Canada 2022
France 2022
Switzerland 2022
Germany 2022
Australia 2022
Nether

,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999,9631172.0,4362626.714,1595229.107,788190.393,1554426.450,464956.846,2285019.608,497456.873,465395.168,184532.987,...,98741.812,39475.706,178944.157,58798.913,148440.319,41661.983,91589.329,60950.313,NaN,84925.900
2000,10250952.0,4423330.247,1682301.743,864938.171,1640509.117,489972.140,2340062.542,531065.413,501507.593,197225.012,...,113573.507,63049.144,198562.939,64536.731,156715.674,47699.612,97008.547,71161.237,NaN,128635.403
2001,10581929.0,4392235.775,1749798.963,895022.950,1706652.728,502541.876,2410461.135,567981.016,534653.181,203856.304,...,134539.224,90906.409,207334.660,69419.634,168861.025,55126.775,104847.848,77621.060,NaN,157475.640
2002,10929108.0,4332958.170,1820370.071,933458.616,1761716.744,500921.187,2438842.475,603034.988,556017.961,209615.124,...,148882.206,133128.130,215469.956,74036.355,181361.825,62412.569,109395.071,84935.412,NaN,190700.479
2003,11456450.0,4328744.001,1924503.401,981201.546,1809244.947,506666.645,2453765.424,648806.489,568969.305,213551.001,...,164559.678,173592.020,224738.559,80867.438,198497.133,68484.731,114780.304,96507.084,NaN,232809.603
2004,12217196.0,4373622.713,2021891.591,1044530.349,1890631.046,521898.821,2510295.105,694863.471,587249.640,223843.884,...,187617.413,214283.445,247537.074,92324.629,214930.203,75573.229,125177.377,109614.735,52540.393,300122.909
2005,13039197.0,4399354.673,2138343.393,1111671.361,1959294.361,541207.835,2538909.443,750440.468,611211.790,235732.184,...,226717.268,250100.630,262774.904,104671.180,221061.913,80889.141,133567.803,123262.185,63091.787,380895.811
2006,13815583.0,4421285.536,2250167.555,1170331.675,2050547.358,575366.059,2646277.005,817588.432,648561.322,250042.138,...,272884.428,292556.843,283706.327,124712.628,241720.218,87155.662,143539.042,139181.031,77536.000,474445.188
2007,14474228.0,4455252.516,2361626.579,1233717.564,2153963.945,612111.996,2773282.946,885855.027,686977.096,258452.642,...,322871.288,326364.570,315030.641,137832.380,258177.650,92155.041,156899.558,156287.426,97147.422,586023.874
2008,14769862.0,4360593.569,2436403.659,1295792.053,2210571.293,638424.397,2825363.481,947539.799,718074.524,267760.829,...,404412.197,368659.321,341045.497,143500.482,268491.457,97553.790,164352.254,173811.798,124512.400,727549.725


In [27]:
indicator = 'SNA_TABLE1' # Gross domestic product
subject = 'B1_GE' # GDP Current Prices
measure = 'VPCOB' # Current Prices, constant PPPs, OECD base year, (US Dollar, Million, 2015)
freq = '' # Yearly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='GDP Current Prices PPP', with_measure=False,
                   filter_measure_on_df=True)

https://stats.oecd.org/SDMX-JSON/data/SNA_TABLE1/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.B1_GE./all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'Eurozone', 'China',
       'India', 'Korea', 'Brazil', 'Saudi Arabia', 'South Africa', 'Mexico',
       'Indonesia', 'Turkiye', 'Poland', 'Chile', 'Greece', 'Hungary',
       'Czechia', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

No data:
Index([], dtype='object')

Missing first:
Argentina 2004

Missing last:
United States 2022
Japan 2022
United Kingdom 2022
Canada 2022
France 2022
Switzerland 2022
Germany 2022
Australia 2022
Nether

## Main Economic Indicators

### Prices indicators

In [28]:
indicator = 'MEI' # Main Economic Indicators
subject = 'CPALTT01' # Customer Price Index - All Items Total
measure = 'IXOB' # Index 2015 = 100
freq = 'M' # Monthly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='Customer Price Index')
df

https://stats.oecd.org/SDMX-JSON/data/MEI/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.CPALTT01.IXOB.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Netherlands', 'Denmark', 'Sweden', 'Spain',
       'Italy', 'Finland', 'Belgium', 'Norway', 'Israel', 'Ireland', 'Austria',
       'Portugal', 'China', 'India', 'Korea', 'Brazil', 'Saudi Arabia',
       'South Africa', 'Mexico', 'Indonesia', 'Turkiye', 'Poland', 'Chile',
       'Greece', 'Hungary', 'Czechia', 'Colombia', 'Russia'],
      dtype='object')

No data:
Index(['Australia', 'New Zealand', 'Eurozone', 'Argentina'], dtype='object')

Missing first:

Missing last:
Japan 2021-06
Russia 2022-03

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,69.3199,99.8727,71.8,72.2939,77.97,91.3508,78.2720,NaN,72.25,72.6511,...,32.3600,7.2400,61.7514,59.1056,68.1409,45.1245,69.3,43.5144,NaN,14.5990
1999-02,69.4043,99.4655,71.9,72.3729,78.21,91.6595,78.3653,NaN,72.61,73.2075,...,32.7724,7.4698,62.1031,59.1481,67.8120,45.7235,69.3,44.2487,NaN,15.2020
1999-03,69.6153,99.5673,72.3,72.6890,78.47,91.7553,78.3653,NaN,73.24,73.5553,...,32.7099,7.7726,62.7361,59.5241,69.3140,46.2643,69.3,44.6647,NaN,15.6261
1999-04,70.1216,100.0764,72.6,73.0840,78.69,91.9084,78.7385,NaN,73.40,73.7640,...,32.4912,8.1509,63.2284,59.7478,70.0350,46.7967,69.4,45.0155,NaN,16.0996
1999-05,70.1216,100.0764,72.8,73.2420,78.70,91.7022,78.7385,NaN,73.42,73.9031,...,32.3975,8.3906,63.6504,59.8184,69.9986,47.1045,69.4,45.2358,NaN,16.4570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05,128.3144,NaN,129.1,124.0453,117.44,106.5078,122.8241,NaN,126.26,116.0000,...,127.4884,499.1078,147.0085,143.7555,114.8052,158.5492,147.6,155.8874,NaN,NaN
2023-06,128.7287,NaN,129.4,124.2033,117.65,106.5755,123.1404,NaN,125.83,116.4000,...,127.6660,518.6753,147.0085,143.5376,115.5154,158.9933,148.1,156.3564,NaN,NaN
2023-07,128.9743,NaN,129.0,124.9144,117.71,106.4244,123.4566,NaN,127.13,118.5000,...,127.9325,567.8915,146.6829,144.0388,114.1926,159.4375,148.8,157.1409,NaN,NaN
2023-08,129.5375,NaN,129.4,125.3885,118.89,106.6516,123.8784,NaN,127.70,117.7000,...,127.9103,619.4946,146.6829,144.2022,114.1563,160.5477,149.1,158.2387,NaN,NaN


In [29]:
indicator = 'MEI' # Main Economic Indicators
subject = 'PIEAFD01' # PPI Manufacture of food products
measure = 'IXOB' # Index 2015 = 100
freq = 'M' # Monthly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='PPI Manufacture of food products')
df

https://stats.oecd.org/SDMX-JSON/data/MEI/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.PIEAFD01.IXOB.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'United Kingdom', 'Canada', 'France', 'Switzerland',
       'Germany', 'Netherlands', 'Denmark', 'Italy', 'Finland', 'Belgium',
       'Norway', 'Ireland', 'Austria', 'Eurozone', 'Poland', 'Chile', 'Greece',
       'Hungary', 'Czechia'],
      dtype='object')

No data:
Index(['Japan', 'Australia', 'Sweden', 'Spain', 'Israel', 'New Zealand',
       'Portugal', 'China', 'India', 'Korea', 'Brazil', 'Saudi Arabia',
       'South Africa', 'Mexico', 'Indonesia', 'Turkiye', 'Colombia',
       'Argentina', 'Russia'],
      dtype='object')

Missing first:
United Kingdom 2008-12
Switzerland 2002-06
Italy 2000-01
Belgium 2001-01
Austria 2000-01
Eurozone 2000-01
Poland 2000-01
Chile 2009-01

Missing last:

,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,62.8054,NaN,NaN,71.6858,80.0,NaN,77.2316,NaN,68.7234,84.2988,...,NaN,NaN,NaN,NaN,67.1366,46.6373,79.8796,NaN,NaN,NaN
1999-02,62.4085,NaN,NaN,71.6858,80.0,NaN,76.9288,NaN,68.4531,83.4963,...,NaN,NaN,NaN,NaN,67.4128,46.7439,79.6299,NaN,NaN,NaN
1999-03,62.3093,NaN,NaN,71.3696,79.9,NaN,76.8512,NaN,68.5252,83.1428,...,NaN,NaN,NaN,NaN,67.7643,46.8505,79.3892,NaN,NaN,NaN
1999-04,61.6644,NaN,NaN,71.4750,79.7,NaN,76.6959,NaN,67.5882,83.5823,...,NaN,NaN,NaN,NaN,68.2498,46.6906,78.8898,NaN,NaN,NaN
1999-05,62.1605,NaN,NaN,71.6858,79.8,NaN,76.5406,NaN,67.5882,83.6682,...,NaN,NaN,NaN,NaN,68.2498,46.4241,78.7293,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09,126.9126,NaN,127.9,131.4592,125.6,105.5398,134.6000,NaN,139.0000,NaN,...,NaN,NaN,143.9,162.4825,112.8000,160.5774,131.0000,NaN,NaN,NaN
2022-10,126.1669,NaN,129.1,132.1971,127.0,105.5848,136.1000,NaN,140.5000,NaN,...,NaN,NaN,145.6,163.9822,114.7000,167.5806,133.5000,NaN,NaN,NaN
2022-11,127.2003,NaN,129.9,132.3026,127.8,105.5999,136.5000,NaN,140.6000,NaN,...,NaN,NaN,146.3,163.0016,115.5000,169.7106,135.4000,NaN,NaN,NaN
2022-12,126.1868,NaN,130.7,131.6700,127.9,106.2940,136.6000,NaN,140.7000,NaN,...,NaN,NaN,146.2,163.3381,116.0000,171.8323,NaN,NaN,NaN,NaN


In [30]:
indicator = 'MEI' # Main Economic Indicators
subject = 'PIEAMP01' # PPI Manufacturing
measure = 'IXOB' # Index 2015 = 100
freq = 'M' # Monthly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='PPI Manufacturing')
df

https://stats.oecd.org/SDMX-JSON/data/MEI/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.PIEAMP01.IXOB.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'United Kingdom', 'Canada', 'France', 'Switzerland',
       'Germany', 'Netherlands', 'Denmark', 'Sweden', 'Spain', 'Italy',
       'Finland', 'Belgium', 'Norway', 'Ireland', 'Austria', 'Portugal',
       'Eurozone', 'Turkiye', 'Poland', 'Chile', 'Greece', 'Hungary',
       'Czechia', 'Colombia'],
      dtype='object')

No data:
Index(['Japan', 'Australia', 'Israel', 'New Zealand', 'China', 'India',
       'Korea', 'Brazil', 'Saudi Arabia', 'South Africa', 'Mexico',
       'Indonesia', 'Argentina', 'Russia'],
      dtype='object')

Missing first:
United Kingdom 2008-12
Switzerland 2002-06
Spain 2000-01
Italy 2000-01
Belgium 2001-01
Austria 2000-01
Eurozone 2000-01
Chile 2003-04

Missing last:


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,68.0140,NaN,NaN,76.9272,79.5,NaN,82.8707,NaN,66.0137,71.2601,...,NaN,6.7958,75.2595,NaN,65.3046,62.4846,85.2870,NaN,NaN,NaN
1999-02,67.8523,NaN,NaN,76.5004,79.5,NaN,82.7845,NaN,65.7417,71.1857,...,NaN,6.9988,75.6859,NaN,65.3046,62.5564,85.6615,NaN,NaN,NaN
1999-03,68.0679,NaN,NaN,77.3540,79.7,NaN,82.8707,NaN,66.4120,71.1857,...,NaN,7.2694,76.1028,NaN,65.7365,62.9151,86.0266,NaN,NaN,NaN
1999-04,68.6607,NaN,NaN,77.3540,80.0,NaN,83.1982,NaN,66.8104,71.5668,...,NaN,7.6237,76.5198,NaN,66.6173,62.2694,86.3075,NaN,NaN,NaN
1999-05,68.8224,NaN,NaN,77.4607,80.1,NaN,83.1120,NaN,67.0144,71.7899,...,NaN,7.9738,76.8515,NaN,66.6850,62.6281,86.4011,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10,136.8898,NaN,140.0,137.9568,127.6,103.2116,126.4000,NaN,146.7000,119.0000,...,NaN,717.1010,139.8000,162.7361,145.1000,155.6634,121.9000,157.5920,NaN,NaN
2022-11,136.2722,NaN,139.4,137.2099,126.7,102.7205,125.8000,NaN,143.4000,118.4000,...,NaN,731.5426,138.3000,158.7487,139.8000,153.8074,120.8000,160.3162,NaN,NaN
2022-12,131.7499,NaN,137.8,135.7162,125.5,102.5178,125.2000,NaN,140.0000,117.4000,...,NaN,748.2840,137.6000,158.6917,131.4000,152.7612,NaN,159.3115,NaN,NaN
2023-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Money indicators

In [31]:
indicator = 'MEI' # Main Economic Indicators
subject = 'MANMM101' # Narrow Money - M1 and components
measure = 'ST' # National Currency
freq = 'M' # Monthly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='Narrow Money M1')
df

https://stats.oecd.org/SDMX-JSON/data/MEI/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.MANMM101.ST.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'Switzerland',
       'Australia', 'Denmark', 'Sweden', 'Norway', 'Israel', 'New Zealand',
       'Eurozone', 'China', 'India', 'Korea', 'Brazil', 'South Africa',
       'Mexico', 'Indonesia', 'Turkiye', 'Poland', 'Chile', 'Hungary',
       'Czechia', 'Russia'],
      dtype='object')

No data:
Index(['France', 'Germany', 'Netherlands', 'Spain', 'Italy', 'Finland',
       'Belgium', 'Ireland', 'Austria', 'Portugal', 'Saudi Arabia', 'Greece',
       'Colombia', 'Argentina'],
      dtype='object')

Missing first:

Missing last:
New Zealand 2018-12

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,1103.9,2.503207e+06,359951.0,203.111,NaN,201287.0,NaN,115.0,NaN,381467.0,...,101.953,3.511547e+03,88635.8,2920.0700,NaN,1822.5840,475.1890,NaN,NaN,177.9
1999-02,1085.2,2.462410e+06,362493.0,200.414,NaN,206120.0,NaN,114.5,NaN,375890.0,...,103.458,3.516057e+03,91943.2,2945.4500,NaN,1845.4674,481.8679,NaN,NaN,180.6
1999-03,1097.5,2.577053e+06,374161.0,199.059,NaN,203007.0,NaN,114.0,NaN,373044.0,...,105.705,4.545328e+03,95283.3,2903.5400,NaN,1884.6005,482.1152,NaN,NaN,174.0
1999-04,1113.6,2.632567e+06,369023.0,203.614,NaN,209720.0,NaN,115.5,NaN,397591.0,...,100.708,3.944113e+03,93989.0,2882.3000,NaN,1881.2429,488.7941,NaN,NaN,195.1
1999-05,1096.4,2.670634e+06,372564.0,207.212,NaN,210485.0,NaN,115.8,NaN,404926.0,...,103.301,3.923985e+03,97092.0,2932.1500,NaN,1989.0889,499.3071,NaN,NaN,205.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,18746.9,1.069616e+07,2317997.0,1505.770,NaN,698014.0,NaN,1625.0,NaN,1565234.7,...,4673.100,3.676969e+06,1548800.0,55688.3942,NaN,31201.7796,4823.1234,NaN,NaN,NaN
2023-05,18501.5,1.073424e+07,2314472.0,1503.487,NaN,683123.0,NaN,1620.7,NaN,1546405.3,...,4623.100,3.523187e+06,1555670.0,55657.7070,NaN,30966.0069,4913.5924,NaN,NaN,NaN
2023-06,18449.4,1.072017e+07,2304395.0,1505.754,NaN,668225.0,NaN,1621.0,NaN,1530061.1,...,4682.800,4.180491e+06,1554380.0,55363.0455,NaN,30465.0099,4907.5512,NaN,NaN,NaN
2023-07,18354.9,1.071008e+07,2283949.0,1510.883,NaN,651319.0,NaN,1617.9,NaN,1544155.3,...,4648.200,4.390129e+06,1571610.0,54720.3053,NaN,30617.4685,4947.9000,NaN,NaN,NaN


In [32]:
indicator = 'MEI' # Main Economic Indicators
subject = 'MABMM301' # Broad Money - M3 and components
measure = 'ST' # National Currency
freq = 'M' # Monthly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='Broad Money M3')
df

https://stats.oecd.org/SDMX-JSON/data/MEI/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.MABMM301.ST.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'Switzerland',
       'Australia', 'Denmark', 'Sweden', 'Norway', 'Israel', 'New Zealand',
       'Eurozone', 'China', 'India', 'Korea', 'Brazil', 'South Africa',
       'Mexico', 'Indonesia', 'Turkiye', 'Poland', 'Chile', 'Hungary',
       'Czechia', 'Russia'],
      dtype='object')

No data:
Index(['France', 'Germany', 'Netherlands', 'Spain', 'Italy', 'Finland',
       'Belgium', 'Ireland', 'Austria', 'Portugal', 'Saudi Arabia', 'Greece',
       'Colombia', 'Argentina'],
      dtype='object')

Missing first:

Missing last:
Czechia 2018-12

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,4409.5,9.415504e+06,817074.0,597.640,NaN,477485.0,NaN,386.1,NaN,538381.0,...,596.541,2.320021e+04,226342.7,31230.5800,NaN,4523.1799,1229.7,NaN,NaN,644.0
1999-02,4418.7,9.364157e+06,824741.0,595.777,NaN,483479.0,NaN,387.4,NaN,525330.0,...,602.666,2.420893e+04,233423.8,31216.2700,NaN,4592.5246,1247.4,NaN,NaN,665.8
1999-03,4455.6,9.431088e+06,832631.0,598.793,NaN,481796.0,NaN,388.7,NaN,518872.0,...,603.325,2.597040e+04,236748.5,31144.8200,NaN,4581.5276,1247.9,NaN,NaN,683.7
1999-04,4501.4,9.481738e+06,835921.0,606.502,NaN,484440.0,NaN,392.8,NaN,550505.0,...,613.140,2.735291e+04,236784.4,31502.1300,NaN,4570.7872,1261.5,NaN,NaN,726.8
1999-05,4464.1,9.506468e+06,844372.0,608.041,NaN,484532.0,NaN,397.9,NaN,547787.0,...,628.260,2.909660e+04,240024.5,32249.6600,NaN,4663.6148,1271.9,NaN,NaN,765.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,20865.9,1.587349e+07,3544161.0,3460.010,NaN,1153875.0,NaN,2868.5,NaN,1889329.4,...,8352.300,9.510649e+06,2147520.0,318732.0483,NaN,40982.4906,NaN,NaN,NaN,NaN
2023-05,20749.1,1.590454e+07,3559219.0,3461.115,NaN,1142752.0,NaN,2874.8,NaN,1878493.6,...,8332.300,9.409262e+06,2157900.0,322092.0552,NaN,40797.6986,NaN,NaN,NaN,NaN
2023-06,20802.6,1.594298e+07,3563149.0,3470.763,NaN,1133624.0,NaN,2871.7,NaN,1864208.0,...,8373.000,1.083449e+07,2157330.0,322524.3287,NaN,39999.2729,NaN,NaN,NaN,NaN
2023-07,20786.2,1.594625e+07,3541354.0,3498.614,NaN,1135752.0,NaN,2896.6,NaN,1880333.5,...,8349.500,1.177296e+07,2173200.0,323314.8085,NaN,40622.9861,NaN,NaN,NaN,NaN


In [33]:
indicator = 'MEI' # Main Economic Indicators
subject = 'XTEXVA01' # Export - Value (goods)
measure = 'CXML' # US Dollars, monthly level
freq = 'M' # Monthly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='Export Value')
df

https://stats.oecd.org/SDMX-JSON/data/MEI/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.XTEXVA01.CXML.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'China', 'India',
       'Korea', 'Brazil', 'Saudi Arabia', 'South Africa', 'Mexico',
       'Indonesia', 'Turkiye', 'Poland', 'Chile', 'Greece', 'Hungary',
       'Czechia', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

No data:
Index(['Eurozone'], dtype='object')

Missing first:
Saudi Arabia 2016-05

Missing last:

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,52.436,30.4931,19.2882,17.4982,24.6222,5.7468,41.6249,4.0003,17.4790,3.8780,...,3.0178,1.8833,2.0090,1.3680,0.8906,1.6671,1.8979,0.7192,1.544,4.154
1999-02,53.279,30.7965,20.0122,18.4034,25.5127,5.9536,44.0156,3.9914,16.8953,3.9074,...,3.1950,2.1945,2.0780,1.3177,0.9383,1.8399,1.8545,0.7621,1.524,4.536
1999-03,60.889,35.9604,23.9591,20.3729,30.3711,7.1199,47.6147,4.6216,20.0086,4.5972,...,3.9530,2.4022,2.4578,1.6650,1.0123,2.2377,2.3025,0.9525,1.998,5.367
1999-04,57.283,33.6024,20.8016,19.3058,26.6563,6.0361,44.7286,4.1257,16.9995,3.7899,...,3.8861,1.9530,2.1643,1.5090,0.9363,1.7904,2.1278,0.8375,2.034,6.017
1999-05,56.489,29.1593,20.8359,19.7162,24.4970,5.8462,42.4310,4.6045,16.6773,4.0280,...,4.0413,2.2256,2.2359,1.3855,0.8913,1.9300,2.2194,0.9297,2.220,4.711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,162.064,62.1409,36.6498,45.8646,51.1139,21.9290,134.2296,29.9100,71.5946,10.0402,...,19.2841,19.2588,30.7124,7.5411,4.3464,12.5904,20.3380,3.7392,5.898,31.787
2023-05,167.128,53.0691,38.4720,48.8413,55.6069,26.5319,142.5291,32.0396,79.8413,11.2594,...,21.7068,21.6376,32.5366,7.8829,4.4018,14.1729,21.9054,4.4979,6.262,37.443
2023-06,167.303,61.8746,39.3680,46.7995,60.9382,27.6218,150.4718,31.7890,80.2621,11.8613,...,20.6013,20.8370,32.4868,7.9564,4.4755,14.6983,22.8058,4.0210,5.415,34.289
2023-07,159.623,61.8523,39.0256,43.9129,54.3924,23.7239,139.2674,29.1243,74.9914,10.9724,...,20.8622,19.9836,29.2538,7.3937,4.7165,12.5711,18.9123,4.0948,6.060,30.804


In [34]:
indicator = 'MEI' # Main Economic Indicators
subject = 'XTIMVA01' # Import - Value (goods)
measure = 'CXML' # US Dollars, monthly level
freq = 'M' # Monthly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='Import Value')
df

https://stats.oecd.org/SDMX-JSON/data/MEI/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.XTIMVA01.CXML.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'China', 'India',
       'Korea', 'Brazil', 'Saudi Arabia', 'South Africa', 'Mexico',
       'Indonesia', 'Turkiye', 'Poland', 'Chile', 'Greece', 'Hungary',
       'Czechia', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

No data:
Index(['Eurozone'], dtype='object')

Missing first:
Saudi Arabia 2014-01

Missing last:

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,71.766,23.8301,24.4352,15.3647,25.1441,5.4742,36.7854,4.5921,16.4811,3.6192,...,1.7722,2.2265,3.1176,1.3849,2.3524,1.8153,1.9064,0.7795,1.906,2.525
1999-02,73.849,22.8159,24.6219,16.7195,24.7966,6.1556,37.8291,4.5470,16.0449,3.5668,...,1.8087,2.7881,3.2110,1.1564,2.4532,2.1136,1.9455,0.8332,1.856,2.773
1999-03,84.072,25.0740,28.0723,19.0737,28.7465,7.4262,41.4894,5.4848,18.7744,4.2634,...,1.9764,3.0451,4.0443,1.2895,3.2292,2.4267,2.5655,0.8935,2.078,3.259
1999-04,79.980,24.9423,25.4327,17.9992,25.2631,5.9515,38.7014,4.9582,15.9826,3.4149,...,2.1219,3.3341,3.6286,1.1815,2.4002,2.2147,2.2376,0.8782,1.873,3.092
1999-05,80.965,22.4442,24.1781,17.9516,24.8980,5.7549,38.9792,5.1194,16.0367,3.4648,...,1.9592,3.4093,3.6985,1.3376,2.4968,2.1426,2.2381,0.7856,1.931,2.723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,250.752,65.4122,56.4619,44.9983,64.5879,19.1415,119.4223,20.5446,70.9241,9.3365,...,15.3476,28.0610,29.9990,6.4711,6.7168,12.1823,18.5152,4.9426,6.091,24.404
2023-05,264.349,63.1280,62.6956,49.3235,67.7369,20.5043,127.8155,24.4003,72.9949,10.9055,...,21.2796,34.1917,31.0291,6.9659,7.6109,12.9301,19.9711,5.4185,7.386,26.070
2023-06,257.887,61.5971,61.4460,49.7581,70.5858,22.2975,126.4401,22.1473,71.4506,10.9556,...,17.1505,26.0624,30.3512,6.3661,7.2888,13.0002,19.8008,5.0152,7.248,25.252
2023-07,255.662,62.3042,58.7622,44.6499,65.2125,20.1405,119.3220,22.8760,69.7951,10.4635,...,19.5703,32.2945,28.3374,6.5799,7.4302,11.8010,18.2307,4.9452,6.761,25.314


In [35]:
indicator = 'MEI' # Main Economic Indicators
subject = 'XTNTVA01' # Net Trade - Value (goods)
measure = 'CXML' # US Dollars, monthly level
freq = 'M' # Monthly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='Net Trade Value')
df

https://stats.oecd.org/SDMX-JSON/data/MEI/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.XTNTVA01.CXML.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'China', 'India',
       'Korea', 'Brazil', 'Saudi Arabia', 'South Africa', 'Mexico',
       'Indonesia', 'Turkiye', 'Poland', 'Chile', 'Greece', 'Hungary',
       'Czechia', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

No data:
Index(['Eurozone'], dtype='object')

Missing first:
Saudi Arabia 2016-05

Missing last:

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,-19.330,6.6630,-5.1470,2.1335,-0.5219,0.2726,4.8394,-0.5919,0.9979,0.2589,...,1.2456,-0.3150,-1.1086,-0.0169,-1.4617,-0.1482,-0.0085,-0.0603,-0.362,1.629
1999-02,-20.570,7.9806,-4.6097,1.6839,0.7161,-0.2020,6.1865,-0.5556,0.8504,0.3407,...,1.3863,-0.5424,-1.1330,0.1613,-1.5149,-0.2737,-0.0911,-0.0710,-0.332,1.763
1999-03,-23.183,10.8864,-4.1132,1.2992,1.6246,-0.3062,6.1252,-0.8632,1.2343,0.3339,...,1.9766,-0.5930,-1.5865,0.3755,-2.2169,-0.1890,-0.2630,0.0590,-0.080,2.108
1999-04,-22.697,8.6601,-4.6311,1.3066,1.3932,0.0845,6.0272,-0.8325,1.0169,0.3751,...,1.7642,-1.3410,-1.4643,0.3275,-1.4639,-0.4243,-0.1098,-0.0408,0.161,2.925
1999-05,-24.476,6.7151,-3.3423,1.7645,-0.4010,0.0912,3.4519,-0.5148,0.6407,0.5633,...,2.0821,-1.1497,-1.4626,0.0479,-1.6055,-0.2126,-0.0187,0.1441,0.289,1.988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,-88.688,-3.2713,-19.8121,0.8663,-13.4740,2.7874,14.8073,9.3654,0.6705,0.7037,...,3.9365,-8.8022,0.7134,1.0700,-2.3705,0.4081,1.8228,-1.2034,-0.193,7.383
2023-05,-97.221,-10.0588,-24.2235,-0.4822,-12.1300,6.0275,14.7137,7.6393,6.8464,0.3539,...,0.4272,-12.5541,1.5075,0.9170,-3.2091,1.2428,1.9344,-0.9207,-1.124,11.373
2023-06,-90.584,0.2775,-22.0780,-2.9586,-9.6477,5.3243,24.0317,9.6417,8.8115,0.9057,...,3.4508,-5.2253,2.1356,1.5903,-2.8133,1.6981,3.0050,-0.9942,-1.833,9.037
2023-07,-96.039,-0.4518,-19.7367,-0.7371,-10.8201,3.5834,19.9454,6.2483,5.1963,0.5089,...,1.2919,-12.3108,0.9165,0.8137,-2.7137,0.7700,0.6817,-0.8504,-0.701,5.490


### Customer sales indicators

In [36]:
indicator = 'MEI' # Main Economic Indicators
subject = 'SLRTTO02' # Total retail sales value
measure = 'IXOBSA' # Index 2015=100, s.a.
freq = 'M' # Monthly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='Retail Sales Value MoM')
df

https://stats.oecd.org/SDMX-JSON/data/MEI/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.SLRTTO02.IXOBSA.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Netherlands', 'Denmark', 'Sweden', 'Spain',
       'Italy', 'Finland', 'Belgium', 'Norway', 'Israel', 'Ireland', 'Austria',
       'Portugal', 'Eurozone', 'South Africa', 'Turkiye', 'Poland', 'Chile',
       'Greece', 'Hungary', 'Czechia', 'Colombia'],
      dtype='object')

No data:
Index(['Australia', 'New Zealand', 'China', 'India', 'Korea', 'Brazil',
       'Saudi Arabia', 'Mexico', 'Indonesia', 'Argentina', 'Russia'],
      dtype='object')

Missing first:
Ireland 2000-01
Turkiye 2010-01
Chile 2005-01

Missing last:
Israel 2017-07
Poland 2018-12
Chile 2016-12
Colombia 2018-12

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,56.8032,95.8237,60.8788,50.6461,57.4952,82.4241,85.2,NaN,88.9096,77.1447,...,NaN,NaN,30.4901,NaN,76.2449,37.1102,59.7235,25.4986,NaN,NaN
1999-02,57.3661,96.8673,60.6460,51.1330,58.7951,83.7597,85.2,NaN,88.5949,78.2957,...,NaN,NaN,31.7056,NaN,76.7515,36.7904,57.1919,26.4986,NaN,NaN
1999-03,57.6621,96.8673,60.9952,51.4001,58.8951,84.4318,86.4,NaN,90.1078,83.1331,...,NaN,NaN,34.4857,NaN,76.1097,38.7518,58.8435,25.9268,NaN,NaN
1999-04,58.0604,97.9109,60.8788,51.3693,58.3951,80.1433,84.2,NaN,90.1281,71.6368,...,NaN,NaN,33.8164,NaN,77.9683,38.9224,58.2806,25.7022,NaN,NaN
1999-05,58.6335,96.4878,61.5773,51.1656,59.2951,84.9224,84.2,NaN,90.5648,78.4956,...,NaN,NaN,35.8033,NaN,77.6120,38.1051,58.7767,26.0471,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,151.3288,108.6555,136.4245,143.7632,137.8885,107.1000,137.5,NaN,138.9232,119.3099,...,NaN,979.5,NaN,NaN,126.3000,204.6162,150.7126,NaN,NaN,NaN
2023-05,152.1273,110.1522,137.4721,143.8978,137.6885,109.6000,139.4,NaN,141.1235,117.8098,...,NaN,1032.7,NaN,NaN,128.6000,202.1074,150.6126,NaN,NaN,NaN
2023-06,152.3246,109.4537,139.1018,144.0957,139.5884,111.2000,138.9,NaN,143.5239,119.3099,...,NaN,1082.5,NaN,NaN,125.0000,204.8169,151.8127,NaN,NaN,NaN
2023-07,152.9344,111.8483,137.5885,144.5505,138.7884,108.7000,139.2,NaN,140.1234,114.0095,...,NaN,1212.4,NaN,NaN,128.9000,204.9172,152.8127,NaN,NaN,NaN


In [37]:
indicator = 'MEI' # Main Economic Indicators
subject = 'SLRTCR03' # Passenger Car Registration
measure = 'IXOBSA' # Index 2015=100, s.a.
freq = 'M' # Monthly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='Passenger Car Registration')
df

https://stats.oecd.org/SDMX-JSON/data/MEI/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.SLRTCR03.IXOBSA.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'Switzerland',
       'Germany', 'Australia', 'Netherlands', 'Denmark', 'Sweden', 'Spain',
       'Italy', 'Finland', 'Belgium', 'Norway', 'Israel', 'Ireland',
       'New Zealand', 'Portugal', 'Eurozone', 'India', 'Korea', 'South Africa',
       'Turkiye', 'Poland', 'Greece', 'Hungary', 'Czechia'],
      dtype='object')

No data:
Index(['France', 'Austria', 'China', 'Brazil', 'Saudi Arabia', 'Mexico',
       'Indonesia', 'Chile', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

Missing first:
Israel 2004-01
India 2001-05
Poland 2003-01
Hungary 2003-01
Czechia 2003-01

Missing last:
Japan 2018-12
Switzerland 2018-12
Australia 2017-12
Netherlands

,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,107.1000,112.1293,85.0050,104.7186,NaN,98.6268,122.0981,107.6567,169.5339,73.7396,...,NaN,22.5595,NaN,NaN,393.4795,NaN,NaN,NaN,NaN,NaN
1999-02,113.1703,112.6388,58.2981,111.0695,NaN,86.7584,112.8924,104.6670,143.5993,73.3293,...,NaN,21.0998,NaN,NaN,296.7068,NaN,NaN,NaN,NaN,NaN
1999-03,111.5099,111.8324,106.0206,107.0831,NaN,109.0537,119.4261,113.2890,142.2885,75.6932,...,NaN,29.5106,NaN,NaN,333.7000,NaN,NaN,NaN,NaN,NaN
1999-04,112.9445,106.2190,95.6814,110.2510,NaN,87.1873,124.5039,113.3380,147.3024,65.4513,...,NaN,44.2336,NaN,NaN,321.4259,NaN,NaN,NaN,NaN,NaN
1999-05,116.3847,109.0316,90.2812,107.2179,NaN,89.0806,118.2200,109.4604,135.5043,70.3815,...,NaN,37.3353,NaN,NaN,333.5663,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05,40.3008,NaN,65.6574,28.9678,NaN,NaN,NaN,NaN,NaN,75.6354,...,NaN,162.0831,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06,40.9302,NaN,70.5006,33.6760,NaN,NaN,NaN,NaN,NaN,83.0337,...,NaN,139.1343,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07,NaN,NaN,76.1129,29.6977,NaN,NaN,NaN,NaN,NaN,81.3575,...,NaN,126.0720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-08,NaN,NaN,77.1411,NaN,NaN,NaN,NaN,NaN,NaN,74.5372,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
indicator = 'MEI' # Main Economic Indicators
subject = 'ODCNPI03' # Permits issued - Residential Buildings
measure = 'IXOBSA' # Index 2015=100, s.a.
freq = 'M' # Monthly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='Permits issued - Residential Buildings')
df

https://stats.oecd.org/SDMX-JSON/data/MEI/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.ODCNPI03.IXOBSA.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['Canada', 'France', 'Germany', 'Australia', 'Netherlands', 'Sweden',
       'Spain', 'Finland', 'Belgium', 'Norway', 'Israel', 'New Zealand',
       'Portugal', 'Eurozone', 'Korea', 'South Africa', 'Turkiye', 'Poland',
       'Chile', 'Greece', 'Hungary', 'Colombia'],
      dtype='object')

No data:
Index(['United States', 'Japan', 'United Kingdom', 'Switzerland', 'Denmark',
       'Italy', 'Ireland', 'Austria', 'China', 'India', 'Brazil',
       'Saudi Arabia', 'Mexico', 'Indonesia', 'Czechia', 'Argentina',
       'Russia'],
      dtype='object')

Missing first:
Poland 2000-01

Missing last:
Netherlands 2018-12
Korea 2018-12
Turkiye 2018-12

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,NaN,NaN,NaN,31.9401,136.9686,NaN,179.8150,64.2354,59.1031,NaN,...,NaN,71.5542,NaN,29.8145,1376.2,171.6857,NaN,23.9943,NaN,NaN
1999-02,NaN,NaN,NaN,35.2742,91.9780,NaN,161.4135,66.2941,46.8075,NaN,...,NaN,97.2070,NaN,31.3052,1300.9,175.3854,NaN,33.4067,NaN,NaN
1999-03,NaN,NaN,NaN,34.7650,81.1964,NaN,176.0147,64.4271,52.5895,NaN,...,NaN,49.6838,NaN,36.7654,1457.2,201.7832,NaN,29.7125,NaN,NaN
1999-04,NaN,NaN,NaN,37.3204,85.7212,NaN,176.2147,63.2817,61.5801,NaN,...,NaN,141.5955,NaN,23.1814,1309.2,225.4812,NaN,30.0431,NaN,NaN
1999-05,NaN,NaN,NaN,37.4368,85.8013,NaN,179.1149,61.5106,67.0556,NaN,...,NaN,68.7000,NaN,29.9610,1310.6,236.7803,NaN,17.9781,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,NaN,NaN,NaN,142.1296,103.8146,NaN,82.2069,61.4808,NaN,NaN,...,NaN,NaN,136.4205,46.1436,382.6,193.7839,NaN,61.3833,NaN,NaN
2023-05,NaN,NaN,NaN,159.9227,84.7174,NaN,83.8070,72.2073,NaN,NaN,...,NaN,NaN,113.1733,51.7688,516.9,202.7831,NaN,85.9307,NaN,NaN
2023-06,NaN,NaN,NaN,157.8710,101.6419,NaN,73.6061,66.6738,NaN,NaN,...,NaN,NaN,117.8062,46.8346,527.0,159.8867,NaN,65.0006,NaN,NaN
2023-07,NaN,NaN,NaN,166.3815,92.4070,NaN,74.2062,65.6482,NaN,NaN,...,NaN,NaN,120.5516,79.9649,NaN,134.0888,NaN,52.4826,NaN,NaN


### Production indicators

In [39]:
indicator = 'MEI' # Main Economic Indicators
subject = 'PRMNTO01' # Total manufacturing
measure = 'IXOBSA' # Index 2015=100, s.a.
freq = 'M' # Monthly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='Total manufacturing')
df

https://stats.oecd.org/SDMX-JSON/data/MEI/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.PRMNTO01.IXOBSA.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Netherlands', 'Denmark', 'Sweden', 'Spain',
       'Italy', 'Finland', 'Belgium', 'Norway', 'Israel', 'Ireland', 'Austria',
       'Portugal', 'Eurozone', 'India', 'Korea', 'Brazil', 'South Africa',
       'Mexico', 'Indonesia', 'Turkiye', 'Poland', 'Chile', 'Greece',
       'Hungary', 'Czechia', 'Colombia', 'Russia'],
      dtype='object')

No data:
Index(['Australia', 'New Zealand', 'China', 'Saudi Arabia', 'Argentina'], dtype='object')

Missing first:
Switzerland 2010-10

Missing last:
Russia 2021-12

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,90.6157,99.8811,77.4878,101.7687,104.4091,NaN,76.3099,NaN,81.5882,89.0118,...,40.5694,37.7517,35.7789,64.9564,136.3168,42.8420,50.7145,72.9814,NaN,47.4824
1999-02,91.4002,99.8811,76.1079,101.8850,103.0191,NaN,74.8057,NaN,79.3047,87.3393,...,50.1690,40.0967,35.8114,65.5043,134.6669,41.0317,50.6539,71.1221,NaN,44.8324
1999-03,91.4026,101.5958,76.8510,101.5594,103.4291,NaN,75.2068,NaN,81.8516,89.1976,...,57.0022,38.5475,38.3142,65.4071,134.3778,42.7322,50.5904,66.7407,NaN,45.2243
1999-04,91.7182,99.6668,76.5325,102.5394,104.5091,NaN,75.7082,NaN,80.0073,83.8086,...,55.8834,41.8816,37.7877,65.1525,134.3540,43.5002,53.3788,69.6242,NaN,44.9443
1999-05,92.4465,101.3815,77.5940,101.9928,105.1191,NaN,76.3099,NaN,80.6221,86.8748,...,56.5898,40.5879,39.1526,65.5299,138.0050,43.0065,53.4623,67.9940,NaN,44.6520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,99.6693,96.6353,113.0473,103.3751,100.1792,NaN,97.0669,NaN,111.0815,157.8000,...,NaN,148.7000,150.8874,99.6610,123.9038,119.5383,118.4901,122.0591,NaN,NaN
2023-05,99.4356,92.0077,112.8350,105.6429,101.4092,NaN,97.2675,NaN,112.1813,156.1000,...,NaN,150.5000,150.8874,99.8760,127.0988,122.7501,121.0899,122.7721,NaN,NaN
2023-06,99.0163,96.3631,115.4887,105.2834,100.3192,NaN,96.3650,NaN,113.1811,165.7000,...,NaN,153.5000,152.3873,99.2764,124.8024,122.2482,122.4898,121.3462,NaN,NaN
2023-07,99.3668,94.0947,114.6395,103.6289,101.0592,NaN,94.5600,NaN,112.4813,149.2000,...,NaN,151.4000,150.5875,98.1579,125.1019,124.0549,117.8902,119.9521,NaN,NaN


In [40]:
indicator = 'MEI' # Main Economic Indicators
subject = 'PRINTO01' # Total industry excluding construction
measure = 'IXOBSA' # Index 2015=100, s.a.
freq = 'M' # Monthly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='Total industry excluding construction')
df

https://stats.oecd.org/SDMX-JSON/data/MEI/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.PRINTO01.IXOBSA.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Netherlands', 'Denmark', 'Sweden', 'Spain',
       'Italy', 'Finland', 'Belgium', 'Norway', 'Israel', 'Ireland', 'Austria',
       'Portugal', 'Eurozone', 'India', 'Korea', 'Brazil', 'Mexico', 'Turkiye',
       'Poland', 'Chile', 'Greece', 'Hungary', 'Czechia', 'Colombia',
       'Russia'],
      dtype='object')

No data:
Index(['Australia', 'New Zealand', 'China', 'Saudi Arabia', 'South Africa',
       'Indonesia', 'Argentina'],
      dtype='object')

Missing first:
Japan 2023-05
Switzerland 2010-10

Missing last:
Mexico 2018-02
Russia 2021-12

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,86.4662,NaN,96.0382,88.6059,102.4391,NaN,75.2063,NaN,87.6438,104.2859,...,NaN,40.7966,41.2715,64.9648,139.1954,47.5216,58.2845,72.9814,NaN,52.5522
1999-02,86.9607,NaN,95.5076,88.5219,102.1291,NaN,73.8062,NaN,88.2087,102.3265,...,NaN,41.0966,41.3667,65.5128,137.6054,45.7615,58.9155,71.1221,NaN,50.9936
1999-03,87.1527,NaN,96.3566,88.1909,102.0591,NaN,74.3062,NaN,89.1501,104.5036,...,NaN,41.9965,43.2552,65.4156,135.6712,47.1696,57.8420,66.7407,NaN,52.2202
1999-04,87.3460,NaN,96.0382,88.7295,102.8791,NaN,74.8062,NaN,87.5497,98.1899,...,NaN,41.3966,43.1906,65.1610,135.5289,47.4629,59.3387,69.6242,NaN,51.6060
1999-05,87.8814,NaN,96.7810,88.5641,103.2791,NaN,75.3063,NaN,87.4556,101.7822,...,NaN,42.7964,44.8868,65.5384,139.1355,47.0523,61.4767,67.9940,NaN,51.7759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,102.2271,NaN,111.6378,109.8663,98.4992,NaN,97.2081,NaN,97.3919,142.0000,...,NaN,144.7879,147.5000,98.0130,120.4795,117.6098,114.4809,122.0591,NaN,NaN
2023-05,101.8890,93.2096,110.8949,109.4746,99.5792,NaN,97.5081,NaN,98.1918,141.0000,...,NaN,146.7878,147.6000,95.2071,120.5794,119.8100,114.9808,122.7721,NaN,NaN
2023-06,101.4736,95.9431,112.9112,109.3071,98.6392,NaN,96.5080,NaN,99.4917,149.5000,...,NaN,148.5876,148.3000,97.9350,118.9810,118.9099,116.5806,121.3462,NaN,NaN
2023-07,102.1920,94.3941,112.1684,109.6894,99.4292,NaN,94.8079,NaN,99.9917,135.9000,...,NaN,148.0877,147.1000,96.5894,121.6783,122.3102,113.5811,119.9521,NaN,NaN


In [41]:
indicator = 'MEI' # Main Economic Indicators
subject = 'PRCNTO01' # Total construction
measure = 'IXOBSA' # Index 2015=100, s.a.
freq = 'M' # Monthly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='Total construction')
df

https://stats.oecd.org/SDMX-JSON/data/MEI/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.PRCNTO01.IXOBSA.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'United Kingdom', 'Canada', 'France', 'Germany',
       'Netherlands', 'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium',
       'Austria', 'Portugal', 'Eurozone', 'China', 'Korea', 'Brazil',
       'South Africa', 'Mexico', 'Poland', 'Chile', 'Hungary', 'Czechia'],
      dtype='object')

No data:
Index(['Japan', 'Switzerland', 'Australia', 'Denmark', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'India', 'Saudi Arabia', 'Indonesia',
       'Turkiye', 'Greece', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

Missing first:
United Kingdom 2010-01
Netherlands 2000-01
Belgium 2000-01
Portugal 2000-01
Korea 2000-01

Missing last:

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,66.0514,NaN,NaN,55.0933,102.44,NaN,105.6695,NaN,NaN,NaN,...,NaN,NaN,75.8437,48.6335,NaN,74.3160,71.1823,NaN,NaN,NaN
1999-02,67.6812,NaN,NaN,55.2141,92.21,NaN,93.7364,NaN,NaN,NaN,...,NaN,NaN,74.8522,49.4655,NaN,73.8276,69.7826,NaN,NaN,NaN
1999-03,68.6986,NaN,NaN,54.4736,98.38,NaN,105.0934,NaN,NaN,NaN,...,NaN,NaN,74.9142,51.1016,NaN,76.0584,69.8038,NaN,NaN,NaN
1999-04,67.9123,NaN,NaN,55.5228,99.70,NaN,102.6245,NaN,NaN,NaN,...,NaN,NaN,77.0210,41.1484,NaN,77.3916,73.4276,NaN,NaN,NaN
1999-05,67.7791,NaN,NaN,56.0506,99.37,NaN,102.7068,NaN,NaN,NaN,...,NaN,NaN,77.5167,43.5075,NaN,78.2232,70.8491,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,170.7894,NaN,117.9608,106.5056,105.18,NaN,135.5113,NaN,142.7119,NaN,...,NaN,NaN,132.8111,43.2516,NaN,147.5000,105.6824,NaN,NaN,NaN
2023-05,174.2713,NaN,117.5101,105.3766,105.86,NaN,136.5114,NaN,147.6123,NaN,...,NaN,NaN,131.1109,56.5713,NaN,149.2000,106.7822,NaN,NaN,NaN
2023-06,175.1211,NaN,119.4254,104.6679,103.10,NaN,131.9110,NaN,146.4122,NaN,...,NaN,NaN,130.3109,50.0714,NaN,147.4000,109.4818,NaN,NaN,NaN
2023-07,176.6848,NaN,118.8621,104.6425,103.74,NaN,134.3112,NaN,147.1123,NaN,...,NaN,NaN,130.3109,77.2725,NaN,159.8000,105.3824,NaN,NaN,NaN
